In [1]:
!pip install -e .

Obtaining file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md
ERROR: file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# py-tidymodels imports
from py_workflows import workflow
from py_parsnip import linear_reg, rand_forest, boost_tree
from py_rsample import initial_split, training, testing
from py_yardstick import metric_set, rmse, mae, r_squared

# Recipe imports
from py_recipes import recipe
from py_recipes.selectors import (
    all_numeric, all_nominal, all_predictors, all_outcomes,
    all_numeric_predictors, all_nominal_predictors,
    contains, starts_with, ends_with, matches  # ← These are now available
)

from py_visualize import plot_forecast, plot_residuals

# Set random seed for reproducibility
np.random.seed(42)

# Recipes Demonstration Notebook

This notebook demonstrates various preprocessing techniques using py-recipes with the same forecasting data.

## Contents:
1. Data loading and splitting
2. Basic recipes
3. Feature engineering recipes
4. Model comparison with different recipes
5. Advanced recipe patterns

# 1. Import Data

In [3]:
# import raw data
raw_data = pd.read_csv('__data/refinery_margins.csv')

In [4]:
df = raw_data

# set date as index
#df = raw_data.set_index('date')

# convert date to datetime
df['date'] = pd.to_datetime(df['date'])
display(df)

,date,country,refinery_kbd,brent,dubai,wti,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,urals_cracking_nw_europe,urals_hydroskimming_nw_europe,es_sider_cracking_med,es_sider_hydroskimming_med,urals_cracking_med,urals_hydroskimming_med,dubai_cracking_singapore,dubai_hydroskimming_singapore,tapis_hydroskimming_singapore,x50_50_hls_lls_cracking_usgc,x30_70_wcs_bakken_cracking_usmc,bakken_coking_usmc
0,2006-01-01,Algeria,450.0000,63.57,58.31,65.48,3.12,-2.51,5.30,-2.18,4.81,-0.17,6.80,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
1,2006-01-01,Denmark,172.9948,63.57,58.31,65.48,3.12,-2.51,5.30,-2.18,4.81,-0.17,6.80,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
2,2006-01-01,Germany,2270.5419,63.57,58.31,65.48,3.12,-2.51,5.30,-2.18,4.81,-0.17,6.80,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
3,2006-01-01,Italy,1859.7058,63.57,58.31,65.48,3.12,-2.51,5.30,-2.18,4.81,-0.17,6.80,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
4,2006-01-01,Netherlands,907.5555,63.57,58.31,65.48,3.12,-2.51,5.30,-2.18,4.81,-0.17,6.80,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,2021-12-01,Norway,224.7000,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
1916,2021-12-01,Romania,226.6026,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
1917,2021-12-01,Russian Federation,5924.5209,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
1918,2021-12-01,Turkey,773.1726,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87


# 2. Train/Test Split

In [5]:
# Create train/test split
split = initial_split(df, prop=0.75, seed=123)
train_data = training(split)
test_data = testing(split)

print(f"Training set: {train_data.shape[0]} rows")
print(f"Test set: {test_data.shape[0]} rows")
display(train_data.head(10))
display(test_data.tail(10))

Training set: 1440 rows
Test set: 480 rows


,date,country,refinery_kbd,brent,dubai,wti,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,urals_cracking_nw_europe,urals_hydroskimming_nw_europe,es_sider_cracking_med,es_sider_hydroskimming_med,urals_cracking_med,urals_hydroskimming_med,dubai_cracking_singapore,dubai_hydroskimming_singapore,tapis_hydroskimming_singapore,x50_50_hls_lls_cracking_usgc,x30_70_wcs_bakken_cracking_usmc,bakken_coking_usmc
0,2006-01-01,Algeria,450.0000,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
1,2006-01-01,Denmark,172.9948,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
2,2006-01-01,Germany,2270.5419,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
3,2006-01-01,Italy,1859.7058,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
4,2006-01-01,Netherlands,907.5555,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
5,2006-01-01,Norway,316.9961,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
6,2006-01-01,Romania,330.4621,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
7,2006-01-01,Russian Federation,4319.2710,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
8,2006-01-01,Turkey,437.9177,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57
9,2006-01-01,United Kingdom,1608.6955,63.57,58.31,65.48,3.12,-2.51,5.3,-2.18,4.81,-0.17,6.8,-1.52,2.47,-3.16,-1.79,-5.68,0.38,2.57


,date,country,refinery_kbd,brent,dubai,wti,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,urals_cracking_nw_europe,urals_hydroskimming_nw_europe,es_sider_cracking_med,es_sider_hydroskimming_med,urals_cracking_med,urals_hydroskimming_med,dubai_cracking_singapore,dubai_hydroskimming_singapore,tapis_hydroskimming_singapore,x50_50_hls_lls_cracking_usgc,x30_70_wcs_bakken_cracking_usmc,bakken_coking_usmc
470,2021-12-01,Algeria,557.0000,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
471,2021-12-01,Denmark,156.5652,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
472,2021-12-01,Germany,1898.9548,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
473,2021-12-01,Italy,1246.4516,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
474,2021-12-01,Netherlands,951.1219,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
475,2021-12-01,Norway,224.7000,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
476,2021-12-01,Romania,226.6026,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
477,2021-12-01,Russian Federation,5924.5209,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
478,2021-12-01,Turkey,773.1726,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87
479,2021-12-01,United Kingdom,1031.5206,74.31,72.76,71.53,5.11,2.89,5.14,0.53,6.52,4.58,5.31,-0.31,8.24,-1.12,3.23,3.45,10.51,11.87


# 3. Define Formula

Using dot notation to include all columns except target and date.

In [6]:
# Define formulas for nested vs global modeling
FORMULA_STR = "refinery_kbd ~ ."  # Dot notation - excludes date automatically
# FORMULA_GLOBAL = "refinery_kbd ~ ."  # Same formula, but fit_global adds 'country'

# Show group structure
print("=== Panel Data Structure ===")
print(f"Total countries: {train_data['country'].nunique()}")
print(f"Countries: {sorted(train_data['country'].unique())}")
print(f"\nObservations per country (train):")
print(train_data.groupby('country').size())
print(f"\nDate range per country (train):")
print(train_data.groupby('country')['date'].agg(['min', 'max']))

=== Panel Data Structure ===
Total countries: 10
Countries: ['Algeria', 'Denmark', 'Germany', 'Italy', 'Netherlands', 'Norway', 'Romania', 'Russian Federation', 'Turkey', 'United Kingdom']

Observations per country (train):
country
Algeria               144
Denmark               144
Germany               144
Italy                 144
Netherlands           144
Norway                144
Romania               144
Russian Federation    144
Turkey                144
United Kingdom        144
dtype: int64

Date range per country (train):
                          min        max
country                                 
Algeria            2006-01-01 2017-12-01
Denmark            2006-01-01 2017-12-01
Germany            2006-01-01 2017-12-01
Italy              2006-01-01 2017-12-01
Netherlands        2006-01-01 2017-12-01
Norway             2006-01-01 2017-12-01
Romania            2006-01-01 2017-12-01
Russian Federation 2006-01-01 2017-12-01
Turkey             2006-01-01 2017-12-01
United King

In [7]:
# Recipe with ANOVA filter (keep top 50% of features)
rec_anova = (
    recipe()
    .step_filter_anova(outcome="refinery_kbd", top_p=0.5, use_pvalue=True)
    .step_normalize(all_numeric_predictors())
)

# Workflow
wf_anova = (
    workflow()
    .add_recipe(rec_anova)
    .add_model(linear_reg().set_engine("sklearn"))
)

# Fit and evaluate
fit_anova = wf_anova.fit_nested(train_data, group_col='country', per_group_prep=True)
fit_anova = fit_anova.evaluate(test_data)

print("✓ ANOVA filter model fitted successfully!")

fit = fit_anova

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

✓ ANOVA filter model fitted successfully!


,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,-0.411900,0.386925,-0.411900,-0.798825,train,linear_reg,,Algeria
1,2006-02-01,Algeria,-0.554261,0.593082,-0.554261,-1.147343,train,linear_reg,,Algeria
2,2006-03-01,Algeria,-0.625441,-0.290919,-0.625441,-0.334522,train,linear_reg,,Algeria
3,2006-04-01,Algeria,-0.401731,-0.879905,-0.401731,0.478174,train,linear_reg,,Algeria
4,2006-05-01,Algeria,-1.235559,-0.676132,-1.235559,-0.559427,train,linear_reg,,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,-1.596937,-0.773184,-1.596937,-0.823754,test,linear_reg,,United Kingdom
1916,2021-09-01,United Kingdom,-2.022208,-0.889848,-2.022208,-1.132360,test,linear_reg,,United Kingdom
1917,2021-10-01,United Kingdom,-2.193722,-0.933038,-2.193722,-1.260684,test,linear_reg,,United Kingdom
1918,2021-11-01,United Kingdom,-1.539314,-1.258085,-1.539314,-0.281229,test,linear_reg,,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,-8.204063e-16,0.072123,-1.137513e-14,1.000000,NaN,linear_reg,,Algeria,-0.142637,0.142637
1,Algeria,brent_cracking_nw_europe,-9.274737e-01,0.318759,-2.909642e+00,0.004233,19.533464,linear_reg,,Algeria,-1.557880,-0.297067
2,Algeria,bakken_coking_usmc,2.252068e+00,0.949397,2.372104e+00,0.019099,173.280945,linear_reg,,Algeria,0.374453,4.129683
3,Algeria,urals_cracking_med,-9.029606e-01,0.363785,-2.482129e+00,0.014287,25.441582,linear_reg,,Algeria,-1.622415,-0.183506
4,Algeria,x30_70_wcs_bakken_cracking_usmc,-2.540120e+00,0.945974,-2.685190e+00,0.008158,172.033662,linear_reg,,Algeria,-4.410965,-0.669275
...,...,...,...,...,...,...,...,...,...,...,...,...
85,United Kingdom,tapis_hydroskimming_singapore,3.771002e-02,0.212340,1.775929e-01,0.859309,14.341151,linear_reg,,United Kingdom,-0.382233,0.457653
86,United Kingdom,urals_cracking_nw_europe,-3.517293e-01,0.233026,-1.509399e+00,0.133534,17.271511,linear_reg,,United Kingdom,-0.812583,0.109124
87,United Kingdom,brent,-1.084727e+00,1.036170,-1.046862e+00,0.297034,341.494573,linear_reg,,United Kingdom,-3.133953,0.964499
88,United Kingdom,dubai,8.343433e-01,1.090210,7.653051e-01,0.445426,378.043628,linear_reg,,United Kingdom,-1.321757,2.990443


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,0.837991,linear_reg,,Algeria
1,Algeria,train,mae,0.614109,linear_reg,,Algeria
2,Algeria,train,mape,369.554022,linear_reg,,Algeria
3,Algeria,train,smape,119.164737,linear_reg,,Algeria
4,Algeria,train,r_squared,0.297771,linear_reg,,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,linear_reg,,United Kingdom
256,United Kingdom,,model_class,LinearRegression,linear_reg,,United Kingdom
257,United Kingdom,train,n_obs_train,144,linear_reg,,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,linear_reg,,United Kingdom


,date,country,brent_cracking_nw_europe,bakken_coking_usmc,urals_cracking_med,x30_70_wcs_bakken_cracking_usmc,urals_cracking_nw_europe,es_sider_cracking_med,brent,dubai,...,urals_hydroskimming_nw_europe,urals_hydroskimming_med,brent_hydroskimming_nw_europe,x50_50_hls_lls_cracking_usgc,dubai_cracking_singapore,es_sider_hydroskimming_med,tapis_hydroskimming_singapore,wti,dubai_hydroskimming_singapore,split
1430,2017-03-01,United Kingdom,NaN,NaN,-0.131079,NaN,-0.052598,NaN,-1.057443,-1.014581,...,NaN,NaN,NaN,1.097357,NaN,0.767560,0.793128,-1.112604,NaN,train
1431,2017-04-01,United Kingdom,NaN,NaN,0.158431,NaN,0.507125,NaN,-1.019196,-0.964241,...,NaN,NaN,NaN,1.457779,NaN,1.454312,1.134777,-1.048256,NaN,train
1432,2017-05-01,United Kingdom,NaN,NaN,0.058354,NaN,0.094280,NaN,-1.099098,-1.047751,...,NaN,NaN,NaN,1.231228,NaN,1.286916,0.783177,-1.159561,NaN,train
1433,2017-06-01,United Kingdom,NaN,NaN,0.133412,NaN,0.356277,NaN,-1.249813,-1.198771,...,NaN,NaN,NaN,1.498971,NaN,1.797689,0.886004,-1.304345,NaN,train
1434,2017-07-01,United Kingdom,NaN,NaN,0.340715,NaN,0.407883,NaN,-1.181650,-1.152333,...,NaN,NaN,NaN,1.423454,NaN,1.621708,0.952343,-1.239997,NaN,train
1435,2017-08-01,United Kingdom,NaN,NaN,0.433644,NaN,0.661942,NaN,-1.080163,-1.043068,...,NaN,NaN,NaN,1.420021,NaN,1.492942,1.204434,-1.179996,NaN,train
1436,2017-09-01,United Kingdom,NaN,NaN,0.912586,NaN,1.277240,NaN,-0.936643,-0.909219,...,NaN,NaN,NaN,1.166009,NaN,1.359884,1.035268,-1.101735,NaN,train
1437,2017-10-01,United Kingdom,NaN,NaN,0.144134,NaN,0.388035,NaN,-0.843487,-0.842099,...,NaN,NaN,NaN,0.953188,NaN,0.844819,0.829615,-1.026517,NaN,train
1438,2017-11-01,United Kingdom,NaN,NaN,-0.356253,NaN,-0.008932,NaN,-0.656040,-0.646983,...,NaN,NaN,NaN,0.754098,NaN,0.613040,0.643864,-0.805211,NaN,train
1439,2017-12-01,United Kingdom,NaN,NaN,-0.677930,NaN,-0.933864,NaN,-0.593936,-0.614594,...,NaN,NaN,NaN,0.551575,NaN,-0.005037,0.534403,-0.749124,NaN,train


,date,country,brent_cracking_nw_europe,bakken_coking_usmc,urals_cracking_med,x30_70_wcs_bakken_cracking_usmc,urals_cracking_nw_europe,es_sider_cracking_med,brent,dubai,...,urals_hydroskimming_nw_europe,urals_hydroskimming_med,brent_hydroskimming_nw_europe,x50_50_hls_lls_cracking_usgc,dubai_cracking_singapore,es_sider_hydroskimming_med,tapis_hydroskimming_singapore,wti,dubai_hydroskimming_singapore,split
0,2018-01-01,Algeria,-0.615285,-0.171070,-0.774433,-0.184520,-1.001348,-0.532182,-0.412926,-0.434697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
1,2018-02-01,Algeria,-0.290507,-0.628337,-0.009556,-0.622057,-0.239172,-0.364974,-0.548116,-0.560742,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
2,2018-03-01,Algeria,-0.948498,-0.458053,-0.002408,-0.448739,-0.540867,-0.238549,-0.509111,-0.541230,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
3,2018-04-01,Algeria,-0.286289,0.008741,0.119115,-0.002719,0.086340,0.140727,-0.312954,-0.340651,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
4,2018-05-01,Algeria,-0.117574,0.599378,-0.052446,0.571775,-0.004962,0.189666,-0.122856,-0.136559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
5,2018-06-01,Algeria,0.046924,0.435047,-0.220433,0.436030,0.094280,0.471065,-0.178143,-0.153730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
6,2018-07-01,Algeria,0.055360,0.140920,0.097670,0.188779,0.300702,0.532238,-0.206545,-0.173241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
7,2018-08-01,Algeria,1.033910,0.439811,0.744599,0.460270,0.753244,1.429450,-0.256152,-0.196265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
8,2018-09-01,Algeria,-0.429698,0.458863,-0.660059,0.443302,-0.552776,-0.116201,-0.039167,-0.005442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
9,2018-10-01,Algeria,-1.281711,-0.025792,-0.502795,0.021522,-0.588503,-0.511790,0.021800,0.070263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test


,bakken_coking_usmc,brent,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,dubai,dubai_cracking_singapore,dubai_hydroskimming_singapore,es_sider_cracking_med,es_sider_hydroskimming_med,tapis_hydroskimming_singapore,urals_cracking_med,urals_cracking_nw_europe,urals_hydroskimming_med,urals_hydroskimming_nw_europe,wti,x30_70_wcs_bakken_cracking_usmc,x50_50_hls_lls_cracking_usgc
Algeria,True,True,True,False,True,False,False,True,False,False,True,True,False,False,False,True,False
Denmark,True,False,True,True,False,False,False,False,False,False,True,True,True,True,False,True,False
Germany,True,False,True,False,False,True,False,True,False,False,True,True,False,False,False,True,True
Italy,True,False,True,False,False,False,False,False,True,False,True,True,True,False,False,True,True
Netherlands,False,True,False,False,True,False,False,False,True,True,False,False,True,True,True,False,True
Norway,True,False,False,True,False,True,False,False,True,False,True,False,True,True,False,False,True
Romania,True,True,True,False,True,False,False,False,False,False,True,True,True,False,False,True,False
Russian Federation,True,False,False,False,False,False,True,False,True,True,True,True,False,False,False,True,True
Turkey,False,True,True,False,True,False,False,True,False,True,True,True,True,False,False,False,False
United Kingdom,False,True,False,False,True,False,False,False,True,True,True,True,False,False,True,False,True


In [8]:
# Recipe with RF importance filter (keep top 5 features)
rec_rf_imp = (
    recipe()
    .step_filter_rf_importance(outcome="refinery_kbd", top_n=5, trees=50)
    .step_normalize(all_numeric_predictors())
)

# Workflow
wf_rf_imp = (
    workflow()
    .add_recipe(rec_rf_imp)
    .add_model(linear_reg().set_engine("sklearn"))
)

# Fit and evaluate
fit_rf_imp = wf_rf_imp.fit_nested(train_data, group_col='country', per_group_prep=True)
fit_rf_imp = fit_rf_imp.evaluate(test_data)

print("✓ RF importance filter model fitted successfully!")

fit = fit_rf_imp

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

✓ RF importance filter model fitted successfully!


,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,-0.411900,0.256843,-0.411900,-0.668743,train,linear_reg,,Algeria
1,2006-02-01,Algeria,-0.554261,0.281241,-0.554261,-0.835502,train,linear_reg,,Algeria
2,2006-03-01,Algeria,-0.625441,0.024619,-0.625441,-0.650060,train,linear_reg,,Algeria
3,2006-04-01,Algeria,-0.401731,-0.128815,-0.401731,-0.272917,train,linear_reg,,Algeria
4,2006-05-01,Algeria,-1.235559,-0.222503,-1.235559,-1.013056,train,linear_reg,,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,-1.596937,-0.759252,-1.596937,-0.837685,test,linear_reg,,United Kingdom
1916,2021-09-01,United Kingdom,-2.022208,-0.834744,-2.022208,-1.187465,test,linear_reg,,United Kingdom
1917,2021-10-01,United Kingdom,-2.193722,-0.819476,-2.193722,-1.374246,test,linear_reg,,United Kingdom
1918,2021-11-01,United Kingdom,-1.539314,-1.127984,-1.539314,-0.411330,test,linear_reg,,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,-1.580487e-15,0.080420,-1.965297e-14,1.000000e+00,NaN,linear_reg,,Algeria,-0.159014,0.159014
1,Algeria,dubai,1.590272e+00,1.519709,1.046432e+00,2.971911e-01,357.105088,linear_reg,,Algeria,-1.414654,4.595199
2,Algeria,brent,-1.773128e+00,1.540318,-1.151144e+00,2.516636e-01,366.856012,linear_reg,,Algeria,-4.818803,1.272548
3,Algeria,bakken_coking_usmc,-4.266878e-01,0.811128,-5.260423e-01,5.997032e-01,101.731220,linear_reg,,Algeria,-2.030535,1.177159
4,Algeria,es_sider_hydroskimming_med,-1.323033e-01,0.108507,-1.219302e+00,2.248097e-01,1.820513,linear_reg,,Algeria,-0.346855,0.082249
5,Algeria,x30_70_wcs_bakken_cracking_usmc,2.591804e-01,0.807593,3.209295e-01,7.487493e-01,100.846292,linear_reg,,Algeria,-1.337676,1.856037
6,Denmark,Intercept,-1.024697e-15,0.083254,-1.230814e-14,1.000000e+00,NaN,linear_reg,,Denmark,-0.164618,0.164618
7,Denmark,dubai,1.414148e+00,1.615417,8.754078e-01,3.828732e-01,376.498378,linear_reg,,Denmark,-1.780021,4.608317
8,Denmark,brent,-1.443599e+00,1.606983,-8.983287e-01,3.705750e-01,372.577591,linear_reg,,Denmark,-4.621093,1.733894
9,Denmark,dubai_cracking_singapore,3.095503e-02,0.115034,2.690947e-01,7.882587e-01,1.909177,linear_reg,,Denmark,-0.196502,0.258412


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,0.944718,linear_reg,,Algeria
1,Algeria,train,mae,0.723473,linear_reg,,Algeria
2,Algeria,train,mape,347.196021,linear_reg,,Algeria
3,Algeria,train,smape,148.871,linear_reg,,Algeria
4,Algeria,train,r_squared,0.107508,linear_reg,,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,linear_reg,,United Kingdom
256,United Kingdom,,model_class,LinearRegression,linear_reg,,United Kingdom
257,United Kingdom,train,n_obs_train,144,linear_reg,,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,linear_reg,,United Kingdom


,date,country,dubai,brent,bakken_coking_usmc,es_sider_hydroskimming_med,x30_70_wcs_bakken_cracking_usmc,refinery_kbd,dubai_cracking_singapore,urals_cracking_med,urals_cracking_nw_europe,x50_50_hls_lls_cracking_usgc,tapis_hydroskimming_singapore,urals_hydroskimming_med,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,es_sider_cracking_med,wti,split
1430,2017-03-01,United Kingdom,-1.014581,-1.057443,NaN,NaN,NaN,-1.261554,NaN,-0.131079,NaN,1.097357,NaN,NaN,NaN,NaN,NaN,-1.112604,train
1431,2017-04-01,United Kingdom,-0.964241,-1.019196,NaN,NaN,NaN,-1.144334,NaN,0.158431,NaN,1.457779,NaN,NaN,NaN,NaN,NaN,-1.048256,train
1432,2017-05-01,United Kingdom,-1.047751,-1.099098,NaN,NaN,NaN,-1.325671,NaN,0.058354,NaN,1.231228,NaN,NaN,NaN,NaN,NaN,-1.159561,train
1433,2017-06-01,United Kingdom,-1.198771,-1.249813,NaN,NaN,NaN,-1.069161,NaN,0.133412,NaN,1.498971,NaN,NaN,NaN,NaN,NaN,-1.304345,train
1434,2017-07-01,United Kingdom,-1.152333,-1.181650,NaN,NaN,NaN,-1.268952,NaN,0.340715,NaN,1.423454,NaN,NaN,NaN,NaN,NaN,-1.239997,train
1435,2017-08-01,United Kingdom,-1.043068,-1.080163,NaN,NaN,NaN,-0.969327,NaN,0.433644,NaN,1.420021,NaN,NaN,NaN,NaN,NaN,-1.179996,train
1436,2017-09-01,United Kingdom,-0.909219,-0.936643,NaN,NaN,NaN,-0.972327,NaN,0.912586,NaN,1.166009,NaN,NaN,NaN,NaN,NaN,-1.101735,train
1437,2017-10-01,United Kingdom,-0.842099,-0.843487,NaN,NaN,NaN,-1.138251,NaN,0.144134,NaN,0.953188,NaN,NaN,NaN,NaN,NaN,-1.026517,train
1438,2017-11-01,United Kingdom,-0.646983,-0.656040,NaN,NaN,NaN,-1.420820,NaN,-0.356253,NaN,0.754098,NaN,NaN,NaN,NaN,NaN,-0.805211,train
1439,2017-12-01,United Kingdom,-0.614594,-0.593936,NaN,NaN,NaN,-1.329370,NaN,-0.677930,NaN,0.551575,NaN,NaN,NaN,NaN,NaN,-0.749124,train


,date,country,dubai,brent,bakken_coking_usmc,es_sider_hydroskimming_med,x30_70_wcs_bakken_cracking_usmc,refinery_kbd,dubai_cracking_singapore,urals_cracking_med,urals_cracking_nw_europe,x50_50_hls_lls_cracking_usgc,tapis_hydroskimming_singapore,urals_hydroskimming_med,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,es_sider_cracking_med,wti,split
0,2018-01-01,Algeria,-0.434697,-0.412926,-0.171070,-0.292615,-0.184520,0.798167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
1,2018-02-01,Algeria,-0.560742,-0.548116,-0.628337,-0.056544,-0.622057,1.123563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
2,2018-03-01,Algeria,-0.541230,-0.509111,-0.458053,0.029300,-0.448739,1.235418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
3,2018-04-01,Algeria,-0.340651,-0.312954,0.008741,0.115144,-0.002719,1.520140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
4,2018-05-01,Algeria,-0.136559,-0.122856,0.599378,0.308294,0.571775,0.289735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
5,2018-06-01,Algeria,-0.153730,-0.178143,0.435047,0.939248,0.436030,0.198218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
6,2018-07-01,Algeria,-0.173241,-0.206545,0.140920,1.063722,0.188779,-0.615273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
7,2018-08-01,Algeria,-0.196265,-0.256152,0.439811,1.801981,0.460270,1.418453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
8,2018-09-01,Algeria,-0.005442,-0.039167,0.458863,0.076515,0.443302,1.784524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
9,2018-10-01,Algeria,0.070263,0.021800,-0.025792,-0.017914,0.021522,1.225249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test


,bakken_coking_usmc,brent,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,dubai,dubai_cracking_singapore,es_sider_cracking_med,es_sider_hydroskimming_med,tapis_hydroskimming_singapore,urals_cracking_med,urals_cracking_nw_europe,urals_hydroskimming_med,wti,x30_70_wcs_bakken_cracking_usmc,x50_50_hls_lls_cracking_usgc
Algeria,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False
Denmark,True,True,False,False,True,True,False,False,False,False,False,False,False,True,False
Germany,True,True,False,False,True,False,False,False,False,True,True,False,False,False,False
Italy,False,True,False,False,True,False,False,False,False,True,False,False,False,True,True
Netherlands,False,True,False,False,True,False,False,False,True,True,False,True,False,False,False
Norway,False,True,True,True,True,True,False,False,False,False,False,False,False,False,False
Romania,False,True,False,False,True,False,False,False,False,True,True,False,False,True,False
Russian Federation,False,True,False,False,True,False,False,False,False,True,False,False,False,True,True
Turkey,False,True,False,False,True,False,True,False,False,True,True,False,False,False,False
United Kingdom,False,True,False,False,True,False,False,False,False,True,False,False,True,False,True


In [9]:
# Recipe with mutual information filter (keep top 6 features)
rec_mi = (
    recipe()
    .step_filter_mutual_info(outcome="refinery_kbd", top_n=6, n_neighbors=3)
    .step_normalize(all_numeric_predictors())
)

# Workflow
wf_mi = (
    workflow()
    .add_recipe(rec_mi)
    .add_model(linear_reg().set_engine("sklearn"))
)

# Fit and evaluate
fit_mi = wf_mi.fit_nested(train_data, group_col='country')
fit_mi = fit_mi.evaluate(test_data)

print("✓ Mutual information filter model fitted successfully!")

outputs_mi, coefs_mi, stats_mi = fit_mi.extract_outputs()

fit = fit_mi

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

✓ Mutual information filter model fitted successfully!


,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,-0.411900,0.109772,-0.411900,-0.521672,train,linear_reg,,Algeria
1,2006-02-01,Algeria,-0.554261,0.230906,-0.554261,-0.785166,train,linear_reg,,Algeria
2,2006-03-01,Algeria,-0.625441,-0.005565,-0.625441,-0.619876,train,linear_reg,,Algeria
3,2006-04-01,Algeria,-0.401731,-0.277377,-0.401731,-0.124354,train,linear_reg,,Algeria
4,2006-05-01,Algeria,-1.235559,-0.064636,-1.235559,-1.170923,train,linear_reg,,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,-1.596937,-0.586369,-1.596937,-1.010568,test,linear_reg,,United Kingdom
1916,2021-09-01,United Kingdom,-2.022208,-0.521040,-2.022208,-1.501168,test,linear_reg,,United Kingdom
1917,2021-10-01,United Kingdom,-2.193722,-0.440753,-2.193722,-1.752969,test,linear_reg,,United Kingdom
1918,2021-11-01,United Kingdom,-1.539314,-0.763890,-1.539314,-0.775424,test,linear_reg,,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,-7.532515e-16,0.080762,-9.326851e-15,1.000000e+00,NaN,linear_reg,,Algeria,-0.159701,0.159701
1,Algeria,dubai,9.068070e-01,1.280867,7.079632e-01,4.801708e-01,251.535022,linear_reg,,Algeria,-1.626020,3.439634
2,Algeria,brent,-8.193715e-01,1.415654,-5.787938e-01,5.636787e-01,307.258692,linear_reg,,Algeria,-3.618729,1.979986
3,Algeria,wti,-1.250121e-01,0.465231,-2.687098e-01,7.885572e-01,33.183859,linear_reg,,Algeria,-1.044974,0.794950
4,Algeria,dubai_hydroskimming_singapore,-1.756799e-02,0.113405,-1.549140e-01,8.771171e-01,1.971757,linear_reg,,Algeria,-0.241818,0.206682
...,...,...,...,...,...,...,...,...,...,...,...,...
65,United Kingdom,dubai,-4.105150e-01,1.012912,-4.052822e-01,6.859023e-01,290.517127,linear_reg,,United Kingdom,-2.413478,1.592448
66,United Kingdom,wti,5.116029e-01,0.283888,1.802132e+00,7.372331e-02,22.820308,linear_reg,,United Kingdom,-0.049765,1.072971
67,United Kingdom,urals_cracking_nw_europe,5.440687e-01,0.086759,6.271041e+00,4.375420e-09,2.131361,linear_reg,,United Kingdom,0.372509,0.715629
68,United Kingdom,x50_50_hls_lls_cracking_usgc,-6.308095e-01,0.158758,-3.973402e+00,1.140476e-04,7.136740,linear_reg,,United Kingdom,-0.944742,-0.316876


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,0.94529,linear_reg,,Algeria
1,Algeria,train,mae,0.694252,linear_reg,,Algeria
2,Algeria,train,mape,347.555967,linear_reg,,Algeria
3,Algeria,train,smape,141.755604,linear_reg,,Algeria
4,Algeria,train,r_squared,0.106426,linear_reg,,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,linear_reg,,United Kingdom
256,United Kingdom,,model_class,LinearRegression,linear_reg,,United Kingdom
257,United Kingdom,train,n_obs_train,144,linear_reg,,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,linear_reg,,United Kingdom


,date,country,dubai,brent,wti,dubai_hydroskimming_singapore,x30_70_wcs_bakken_cracking_usmc,x50_50_hls_lls_cracking_usgc,refinery_kbd,urals_cracking_med,es_sider_hydroskimming_med,urals_hydroskimming_med,es_sider_cracking_med,urals_cracking_nw_europe,bakken_coking_usmc,brent_hydroskimming_nw_europe,brent_cracking_nw_europe,tapis_hydroskimming_singapore,split
1430,2017-03-01,United Kingdom,-1.014581,-1.057443,-1.112604,NaN,NaN,1.097357,-1.261554,NaN,NaN,NaN,NaN,-0.052598,NaN,NaN,NaN,0.793128,train
1431,2017-04-01,United Kingdom,-0.964241,-1.019196,-1.048256,NaN,NaN,1.457779,-1.144334,NaN,NaN,NaN,NaN,0.507125,NaN,NaN,NaN,1.134777,train
1432,2017-05-01,United Kingdom,-1.047751,-1.099098,-1.159561,NaN,NaN,1.231228,-1.325671,NaN,NaN,NaN,NaN,0.094280,NaN,NaN,NaN,0.783177,train
1433,2017-06-01,United Kingdom,-1.198771,-1.249813,-1.304345,NaN,NaN,1.498971,-1.069161,NaN,NaN,NaN,NaN,0.356277,NaN,NaN,NaN,0.886004,train
1434,2017-07-01,United Kingdom,-1.152333,-1.181650,-1.239997,NaN,NaN,1.423454,-1.268952,NaN,NaN,NaN,NaN,0.407883,NaN,NaN,NaN,0.952343,train
1435,2017-08-01,United Kingdom,-1.043068,-1.080163,-1.179996,NaN,NaN,1.420021,-0.969327,NaN,NaN,NaN,NaN,0.661942,NaN,NaN,NaN,1.204434,train
1436,2017-09-01,United Kingdom,-0.909219,-0.936643,-1.101735,NaN,NaN,1.166009,-0.972327,NaN,NaN,NaN,NaN,1.277240,NaN,NaN,NaN,1.035268,train
1437,2017-10-01,United Kingdom,-0.842099,-0.843487,-1.026517,NaN,NaN,0.953188,-1.138251,NaN,NaN,NaN,NaN,0.388035,NaN,NaN,NaN,0.829615,train
1438,2017-11-01,United Kingdom,-0.646983,-0.656040,-0.805211,NaN,NaN,0.754098,-1.420820,NaN,NaN,NaN,NaN,-0.008932,NaN,NaN,NaN,0.643864,train
1439,2017-12-01,United Kingdom,-0.614594,-0.593936,-0.749124,NaN,NaN,0.551575,-1.329370,NaN,NaN,NaN,NaN,-0.933864,NaN,NaN,NaN,0.534403,train


,date,country,dubai,brent,wti,dubai_hydroskimming_singapore,x30_70_wcs_bakken_cracking_usmc,x50_50_hls_lls_cracking_usgc,refinery_kbd,urals_cracking_med,es_sider_hydroskimming_med,urals_hydroskimming_med,es_sider_cracking_med,urals_cracking_nw_europe,bakken_coking_usmc,brent_hydroskimming_nw_europe,brent_cracking_nw_europe,tapis_hydroskimming_singapore,split
0,2018-01-01,Algeria,-0.434697,-0.412926,-0.499991,0.579561,-0.184520,0.149961,0.798167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
1,2018-02-01,Algeria,-0.560742,-0.548116,-0.565209,1.048560,-0.622057,0.870806,1.123563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
2,2018-03-01,Algeria,-0.541230,-0.509111,-0.539557,0.902313,-0.448739,0.661418,1.235418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
3,2018-04-01,Algeria,-0.340651,-0.312954,-0.384773,0.297152,-0.002719,0.304428,1.520140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
4,2018-05-01,Algeria,-0.136559,-0.122856,-0.225642,0.413141,0.571775,0.441732,0.289735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
5,2018-06-01,Algeria,-0.153730,-0.178143,-0.332599,0.166034,0.436030,0.692311,0.198218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
6,2018-07-01,Algeria,-0.173241,-0.206545,-0.188250,0.836754,0.188779,0.826182,-0.615273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
7,2018-08-01,Algeria,-0.196265,-0.256152,-0.312164,1.088904,0.460270,1.430319,1.418453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
8,2018-09-01,Algeria,-0.005442,-0.039167,-0.215642,0.372797,0.443302,0.520681,1.784524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
9,2018-10-01,Algeria,0.070263,0.021800,-0.192163,0.806496,0.021522,0.342187,1.225249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test


,bakken_coking_usmc,brent,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,dubai,dubai_hydroskimming_singapore,es_sider_cracking_med,es_sider_hydroskimming_med,tapis_hydroskimming_singapore,urals_cracking_med,urals_cracking_nw_europe,urals_hydroskimming_med,wti,x30_70_wcs_bakken_cracking_usmc,x50_50_hls_lls_cracking_usgc
Algeria,False,True,False,False,True,True,False,False,False,False,False,False,True,True,True
Denmark,False,True,False,False,True,False,False,True,False,True,False,True,True,False,False
Germany,False,False,False,False,True,False,True,True,False,True,True,False,True,False,False
Italy,True,True,False,False,True,False,False,False,False,True,False,False,True,True,False
Netherlands,False,False,False,False,True,False,True,True,False,True,False,False,True,False,True
Norway,False,True,False,False,True,False,True,False,False,True,True,False,True,False,False
Romania,False,True,False,True,True,False,False,False,False,True,True,False,True,False,False
Russian Federation,False,True,False,False,True,False,False,False,False,True,False,True,True,False,True
Turkey,False,True,True,False,True,False,True,False,True,False,True,False,False,False,False
United Kingdom,False,True,False,False,True,False,False,False,True,False,True,False,True,False,True


# 26. Advanced Dimensionality Reduction

## ICA, Kernel PCA, and PLS

Advanced methods for feature extraction and dimensionality reduction.

In [10]:
# Recipe with ICA (finds independent signals)
rec_ica = (
    recipe()
    .step_normalize(all_numeric_predictors())
    .step_ica(all_numeric_predictors(), num_comp=5)
)

# Workflow
wf_ica = (
    workflow()
    .add_recipe(rec_ica)
    .add_model(linear_reg().set_engine("sklearn"))
)

# Fit and evaluate
fit_ica = wf_ica.fit_nested(train_data, group_col='country', per_group_prep=True)
fit_ica = fit_ica.evaluate(test_data)

print("✓ ICA model fitted successfully!")

fit = fit_ica

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

✓ ICA model fitted successfully!


,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,450.0000,473.101782,450.0000,-23.101782,train,linear_reg,,Algeria
1,2006-02-01,Algeria,436.0000,462.481706,436.0000,-26.481706,train,linear_reg,,Algeria
2,2006-03-01,Algeria,429.0000,457.071074,429.0000,-28.071074,train,linear_reg,,Algeria
3,2006-04-01,Algeria,451.0000,455.323074,451.0000,-4.323074,train,linear_reg,,Algeria
4,2006-05-01,Algeria,369.0000,466.124077,369.0000,-97.124077,train,linear_reg,,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,1027.1426,1271.447655,1027.1426,-244.305055,test,linear_reg,,United Kingdom
1916,2021-09-01,United Kingdom,943.2540,1237.562798,943.2540,-294.308798,test,linear_reg,,United Kingdom
1917,2021-10-01,United Kingdom,909.4213,1222.475816,909.4213,-313.054516,test,linear_reg,,United Kingdom
1918,2021-11-01,United Kingdom,1038.5093,1181.774049,1038.5093,-143.264749,test,linear_reg,,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,490.506944,7.367874,66.573746,0.000000e+00,NaN,linear_reg,,Algeria,475.938422,505.075467
1,Algeria,IC1,-23.483354,7.367874,-3.187263,1.776823e-03,1.0,linear_reg,,Algeria,-38.051877,-8.914831
2,Algeria,IC2,-0.933196,7.367874,-0.126657,8.993959e-01,1.0,linear_reg,,Algeria,-15.501719,13.635326
3,Algeria,IC3,-5.090938,7.367874,-0.690964,4.907486e-01,1.0,linear_reg,,Algeria,-19.659461,9.477584
4,Algeria,IC4,-2.269815,7.367874,-0.308069,7.584942e-01,1.0,linear_reg,,Algeria,-16.838338,12.298707
5,Algeria,IC5,-39.953314,7.367874,-5.422638,2.552731e-07,1.0,linear_reg,,Algeria,-54.521837,-25.384791
6,Denmark,Intercept,151.955490,1.453689,104.530984,0.000000e+00,NaN,linear_reg,,Denmark,149.081106,154.829873
7,Denmark,IC1,-0.802750,1.453689,-0.552216,5.816937e-01,1.0,linear_reg,,Denmark,-3.677133,2.071634
8,Denmark,IC2,-2.053564,1.453689,-1.412658,1.600077e-01,1.0,linear_reg,,Denmark,-4.927948,0.820819
9,Denmark,IC3,-4.097959,1.453689,-2.819007,5.526363e-03,1.0,linear_reg,,Denmark,-6.972342,-1.223575


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,86.552917,linear_reg,,Algeria
1,Algeria,train,mae,63.32577,linear_reg,,Algeria
2,Algeria,train,mape,12.760989,linear_reg,,Algeria
3,Algeria,train,smape,14.939782,linear_reg,,Algeria
4,Algeria,train,r_squared,0.225381,linear_reg,,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,linear_reg,,United Kingdom
256,United Kingdom,,model_class,LinearRegression,linear_reg,,United Kingdom
257,United Kingdom,train,n_obs_train,144,linear_reg,,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,linear_reg,,United Kingdom


,date,country,refinery_kbd,IC1,IC2,IC3,IC4,IC5,split
1430,2017-03-01,United Kingdom,1093.3000,-1.141516,0.314402,0.139459,0.572525,-0.489402,train
1431,2017-04-01,United Kingdom,1116.4227,-1.463619,0.137784,0.369469,0.565966,0.176313,train
1432,2017-05-01,United Kingdom,1080.6523,-1.463006,0.399611,-0.028197,0.451818,0.141836,train
1433,2017-06-01,United Kingdom,1131.2513,-1.769771,0.558252,0.183692,0.669823,0.411606,train
1434,2017-07-01,United Kingdom,1091.8406,-1.439938,0.659002,0.470319,0.843864,0.470702,train
1435,2017-08-01,United Kingdom,1150.9445,-1.177714,0.574443,0.880005,0.959529,0.511135,train
1436,2017-09-01,United Kingdom,1150.3527,-0.574368,0.708252,0.791737,1.345762,0.947923,train
1437,2017-10-01,United Kingdom,1117.6226,-0.600606,0.936919,0.836114,0.765319,0.247745,train
1438,2017-11-01,United Kingdom,1061.8833,-0.726265,0.761411,0.731577,0.325860,-0.045733,train
1439,2017-12-01,United Kingdom,1079.9226,-0.579870,0.571814,0.887765,0.168780,-1.110631,train


,date,country,refinery_kbd,IC1,IC2,IC3,IC4,IC5,split
0,2018-01-01,Algeria,569.0,-0.333641,0.387385,0.531656,0.016655,-1.140184,test
1,2018-02-01,Algeria,601.0,-0.682442,-0.109648,0.429548,0.541580,-0.923453,test
2,2018-03-01,Algeria,612.0,-0.454546,0.110086,0.479809,0.478843,-1.026394,test
3,2018-04-01,Algeria,640.0,-0.017433,0.166166,0.166914,0.420639,-0.230757,test
4,2018-05-01,Algeria,519.0,-0.213019,0.405843,0.502362,0.073177,0.065680,test
5,2018-06-01,Algeria,510.0,-0.985049,0.355829,0.017734,-0.266134,0.540435,test
6,2018-07-01,Algeria,430.0,-1.038936,0.313253,0.001786,-0.026776,0.642727,test
7,2018-08-01,Algeria,630.0,-1.412204,-0.176683,0.517845,0.191691,1.048572,test
8,2018-09-01,Algeria,666.0,-0.547742,0.291417,0.733654,-0.332087,-0.527052,test
9,2018-10-01,Algeria,611.0,-0.554627,0.621788,0.204911,-0.246861,-0.406675,test


,IC1,IC2,IC3,IC4,IC5
Algeria,True,True,True,True,True
Denmark,True,True,True,True,True
Germany,True,True,True,True,True
Italy,True,True,True,True,True
Netherlands,True,True,True,True,True
Norway,True,True,True,True,True
Romania,True,True,True,True,True
Russian Federation,True,True,True,True,True
Turkey,True,True,True,True,True
United Kingdom,True,True,True,True,True


In [11]:
# Recipe with Kernel PCA (non-linear dimensionality reduction)
rec_kpca = (
    recipe()
    .step_normalize(all_numeric_predictors())
    .step_kpca(all_numeric_predictors(), num_comp=5, kernel='rbf')
)

# Workflow
wf_kpca = (
    workflow()
    .add_recipe(rec_kpca)
    .add_model(linear_reg().set_engine("sklearn"))
)

# Fit and evaluate
fit_kpca = wf_kpca.fit_nested(train_data, group_col='country', per_group_prep=True)
fit_kpca = fit_kpca.evaluate(test_data)

print("✓ Kernel PCA model fitted successfully!")

fit = fit_kpca

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

✓ Kernel PCA model fitted successfully!


,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,450.0000,458.343524,450.0000,-8.343524,train,linear_reg,,Algeria
1,2006-02-01,Algeria,436.0000,464.392259,436.0000,-28.392259,train,linear_reg,,Algeria
2,2006-03-01,Algeria,429.0000,494.714518,429.0000,-65.714518,train,linear_reg,,Algeria
3,2006-04-01,Algeria,451.0000,470.366834,451.0000,-19.366834,train,linear_reg,,Algeria
4,2006-05-01,Algeria,369.0000,454.556015,369.0000,-85.556015,train,linear_reg,,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,1027.1426,1317.274138,1027.1426,-290.131538,test,linear_reg,,United Kingdom
1916,2021-09-01,United Kingdom,943.2540,1265.732130,943.2540,-322.478130,test,linear_reg,,United Kingdom
1917,2021-10-01,United Kingdom,909.4213,1311.639186,909.4213,-402.217886,test,linear_reg,,United Kingdom
1918,2021-11-01,United Kingdom,1038.5093,1348.279999,1038.5093,-309.770699,test,linear_reg,,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,490.506944,7.350504,66.731062,0.000000e+00,NaN,linear_reg,,Algeria,475.972767,505.041122
1,Algeria,KPC1,29.396653,19.595804,1.500150,1.358598e-01,1.0,linear_reg,,Algeria,-9.350199,68.143505
2,Algeria,KPC2,42.929123,23.647932,1.815344,7.164237e-02,1.0,linear_reg,,Algeria,-3.830017,89.688262
3,Algeria,KPC3,106.662817,31.029942,3.437416,7.765754e-04,1.0,linear_reg,,Algeria,45.307204,168.018431
4,Algeria,KPC4,-159.245242,33.097966,-4.811330,3.878521e-06,1.0,linear_reg,,Algeria,-224.689967,-93.800516
5,Algeria,KPC5,25.790098,37.057670,0.695945,4.876337e-01,1.0,linear_reg,,Algeria,-47.484165,99.064361
6,Denmark,Intercept,151.955490,1.474974,103.022471,0.000000e+00,NaN,linear_reg,,Denmark,149.039018,154.871961
7,Denmark,KPC1,6.263565,3.932153,1.592910,1.134684e-01,1.0,linear_reg,,Denmark,-1.511495,14.038625
8,Denmark,KPC2,6.367858,4.745265,1.341939,1.818194e-01,1.0,linear_reg,,Denmark,-3.014971,15.750688
9,Denmark,KPC3,-0.186825,6.226561,-0.030005,9.761068e-01,1.0,linear_reg,,Denmark,-12.498627,12.124977


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,86.348872,linear_reg,,Algeria
1,Algeria,train,mae,59.85092,linear_reg,,Algeria
2,Algeria,train,mape,11.920365,linear_reg,,Algeria
3,Algeria,train,smape,14.107512,linear_reg,,Algeria
4,Algeria,train,r_squared,0.229029,linear_reg,,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,linear_reg,,United Kingdom
256,United Kingdom,,model_class,LinearRegression,linear_reg,,United Kingdom
257,United Kingdom,train,n_obs_train,144,linear_reg,,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,linear_reg,,United Kingdom


,date,country,refinery_kbd,KPC1,KPC2,KPC3,KPC4,KPC5,split
1430,2017-03-01,United Kingdom,1093.3000,0.541806,0.203555,0.080398,-0.347199,-0.009635,train
1431,2017-04-01,United Kingdom,1116.4227,0.670843,-0.160706,0.216901,-0.140518,-0.088593,train
1432,2017-05-01,United Kingdom,1080.6523,0.611374,0.010715,0.160842,-0.229154,-0.098052,train
1433,2017-06-01,United Kingdom,1131.2513,0.621244,-0.206915,0.319672,-0.106651,-0.112821,train
1434,2017-07-01,United Kingdom,1091.8406,0.624361,-0.264452,0.311305,-0.055784,-0.107188,train
1435,2017-08-01,United Kingdom,1150.9445,0.600321,-0.339540,0.291206,0.042259,-0.106537,train
1436,2017-09-01,United Kingdom,1150.3527,0.510990,-0.372107,0.258227,0.165957,-0.083970,train
1437,2017-10-01,United Kingdom,1117.6226,0.599273,-0.173642,0.066617,-0.139647,-0.079865,train
1438,2017-11-01,United Kingdom,1061.8833,0.490691,0.007920,-0.096066,-0.273868,-0.097656,train
1439,2017-12-01,United Kingdom,1079.9226,0.159857,0.270292,-0.124411,-0.395482,0.002691,train


,date,country,refinery_kbd,KPC1,KPC2,KPC3,KPC4,KPC5,split
0,2018-01-01,Algeria,569.0,-0.060257,0.358340,-0.116427,-0.320165,0.014640,test
1,2018-02-01,Algeria,601.0,0.361759,0.270436,-0.067277,-0.305869,0.103425,test
2,2018-03-01,Algeria,612.0,0.240949,0.313905,-0.109300,-0.315981,0.104375,test
3,2018-04-01,Algeria,640.0,0.232445,0.268114,-0.304309,-0.220028,0.095041,test
4,2018-05-01,Algeria,519.0,0.226017,0.083979,-0.367538,-0.230104,-0.065491,test
5,2018-06-01,Algeria,510.0,0.329209,0.069737,-0.230128,-0.144188,-0.209149,test
6,2018-07-01,Algeria,430.0,0.451736,-0.011954,-0.120748,-0.107227,-0.171109,test
7,2018-08-01,Algeria,630.0,0.542326,-0.357070,0.124298,0.168177,-0.168339,test
8,2018-09-01,Algeria,666.0,0.011121,0.168400,-0.298454,-0.335479,-0.093600,test
9,2018-10-01,Algeria,611.0,-0.012307,0.288337,-0.185197,-0.263291,-0.143964,test


,KPC1,KPC2,KPC3,KPC4,KPC5
Algeria,True,True,True,True,True
Denmark,True,True,True,True,True
Germany,True,True,True,True,True
Italy,True,True,True,True,True
Netherlands,True,True,True,True,True
Norway,True,True,True,True,True
Romania,True,True,True,True,True
Russian Federation,True,True,True,True,True
Turkey,True,True,True,True,True
United Kingdom,True,True,True,True,True


In [12]:
# Recipe with PLS (supervised dimensionality reduction)
rec_pls = (
    recipe()
    .step_normalize(all_numeric_predictors())
    .step_pls(all_numeric_predictors(), outcome="refinery_kbd", num_comp=5)
)

# Workflow
wf_pls = (
    workflow()
    .add_recipe(rec_pls)
    .add_model(linear_reg().set_engine("sklearn"))
)

# Fit and evaluate
fit_pls = wf_pls.fit_nested(train_data, group_col='country', per_group_prep=True)
fit_pls = fit_pls.evaluate(test_data)

print("✓ PLS model fitted successfully!")

fit = fit_pls

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

✓ PLS model fitted successfully!


,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,450.0000,442.990798,450.0000,7.009202,train,linear_reg,,Algeria
1,2006-02-01,Algeria,436.0000,467.688053,436.0000,-31.688053,train,linear_reg,,Algeria
2,2006-03-01,Algeria,429.0000,467.902732,429.0000,-38.902732,train,linear_reg,,Algeria
3,2006-04-01,Algeria,451.0000,443.593545,451.0000,7.406455,train,linear_reg,,Algeria
4,2006-05-01,Algeria,369.0000,500.903187,369.0000,-131.903187,train,linear_reg,,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,1027.1426,1076.473114,1027.1426,-49.330514,test,linear_reg,,United Kingdom
1916,2021-09-01,United Kingdom,943.2540,1086.384107,943.2540,-143.130107,test,linear_reg,,United Kingdom
1917,2021-10-01,United Kingdom,909.4213,996.586780,909.4213,-87.165480,test,linear_reg,,United Kingdom
1918,2021-11-01,United Kingdom,1038.5093,880.309527,1038.5093,158.199773,test,linear_reg,,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,490.506944,6.859131,71.511527,0.000000,NaN,linear_reg,,Algeria,476.932925,504.080963
1,Algeria,brent,-571.392054,255.374829,-2.237464,0.027013,1386.174670,linear_reg,,Algeria,-1076.771334,-66.012774
2,Algeria,dubai,296.066341,252.167916,1.174084,0.242576,1351.579010,linear_reg,,Algeria,-202.966552,795.099234
3,Algeria,wti,237.869652,95.165293,2.499542,0.013721,192.494779,linear_reg,,Algeria,49.540335,426.198968
4,Algeria,brent_cracking_nw_europe,-214.524964,120.260818,-1.783831,0.076858,307.404407,linear_reg,,Algeria,-452.517588,23.467660
...,...,...,...,...,...,...,...,...,...,...,...,...
175,United Kingdom,dubai_hydroskimming_singapore,145.137232,42.472072,3.417239,0.000852,23.894469,linear_reg,,United Kingdom,61.086251,229.188213
176,United Kingdom,tapis_hydroskimming_singapore,32.121054,43.603549,0.736661,0.462698,25.184548,linear_reg,,United Kingdom,-54.169087,118.411195
177,United Kingdom,x50_50_hls_lls_cracking_usgc,-60.306191,42.357870,-1.423730,0.156996,23.766143,linear_reg,,United Kingdom,-144.131170,23.518789
178,United Kingdom,x30_70_wcs_bakken_cracking_usmc,1301.429852,356.774923,3.647762,0.000386,1686.083705,linear_reg,,United Kingdom,595.382768,2007.476935


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,76.993557,linear_reg,,Algeria
1,Algeria,train,mae,54.947692,linear_reg,,Algeria
2,Algeria,train,mape,11.173539,linear_reg,,Algeria
3,Algeria,train,smape,13.515596,linear_reg,,Algeria
4,Algeria,train,r_squared,0.387038,linear_reg,,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,linear_reg,,United Kingdom
256,United Kingdom,,model_class,LinearRegression,linear_reg,,United Kingdom
257,United Kingdom,train,n_obs_train,144,linear_reg,,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,linear_reg,,United Kingdom


,date,country,refinery_kbd,brent,dubai,wti,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,urals_cracking_nw_europe,urals_hydroskimming_nw_europe,...,es_sider_hydroskimming_med,urals_cracking_med,urals_hydroskimming_med,dubai_cracking_singapore,dubai_hydroskimming_singapore,tapis_hydroskimming_singapore,x50_50_hls_lls_cracking_usgc,x30_70_wcs_bakken_cracking_usmc,bakken_coking_usmc,split
1430,2017-03-01,United Kingdom,1093.3000,-1.057443,-1.014581,-1.112604,-0.311597,0.378881,-0.052598,0.769716,...,0.767560,-0.131079,1.016494,0.008706,0.635034,0.793128,1.097357,-0.332386,-0.356835,train
1431,2017-04-01,United Kingdom,1116.4227,-1.019196,-0.964241,-1.048256,0.519328,1.098069,0.507125,1.207120,...,1.454312,0.158431,1.270942,0.195133,0.801453,1.134777,1.457779,0.038490,0.043274,train
1432,2017-05-01,United Kingdom,1080.6523,-1.099098,-1.047751,-1.159561,0.245165,1.007605,0.094280,0.932051,...,1.286916,0.058354,1.364239,-0.326861,0.534173,0.783177,1.231228,-0.077863,-0.102004,train
1433,2017-06-01,United Kingdom,1131.2513,-1.249813,-1.198771,-1.304345,0.637429,1.563958,0.356277,1.477679,...,1.797689,0.133412,1.792559,0.143865,1.341055,0.886004,1.498971,-0.077863,-0.122247,train
1434,2017-07-01,United Kingdom,1091.8406,-1.181650,-1.152333,-1.239997,0.734440,1.532296,0.407883,1.608449,...,1.621708,0.340715,1.834967,0.507397,1.608334,0.952343,1.423454,0.181507,0.139729,train
1435,2017-08-01,United Kingdom,1150.9445,-1.080163,-1.043068,-1.179996,1.152011,1.654422,0.661942,1.594921,...,1.492942,0.433644,1.639891,0.945498,1.764667,1.204434,1.420021,0.473602,0.470771,train
1436,2017-09-01,United Kingdom,1150.3527,-0.936643,-0.909219,-1.101735,1.312291,1.631806,1.277240,2.059381,...,1.359884,0.912586,1.970672,1.257762,2.011775,1.035268,1.166009,0.597227,0.602950,train
1437,2017-10-01,United Kingdom,1117.6226,-0.843487,-0.842099,-1.026517,0.363266,0.917141,0.388035,1.283778,...,0.844819,0.144134,1.186126,0.600610,1.361227,0.829615,0.953188,0.731761,0.768472,train
1438,2017-11-01,United Kingdom,1061.8833,-0.656040,-0.646983,-0.805211,0.329523,0.690981,-0.008932,0.688548,...,0.613040,-0.356253,0.562730,0.251061,1.119162,0.643864,0.754098,0.540263,0.533884,train
1439,2017-12-01,United Kingdom,1079.9226,-0.593936,-0.614594,-0.749124,-0.488748,-0.123194,-0.933864,-0.321540,...,-0.005037,-0.677930,0.066557,0.381559,1.063689,0.534403,0.551575,0.055458,0.045656,train


,date,country,refinery_kbd,brent,dubai,wti,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,urals_cracking_nw_europe,urals_hydroskimming_nw_europe,...,es_sider_hydroskimming_med,urals_cracking_med,urals_hydroskimming_med,dubai_cracking_singapore,dubai_hydroskimming_singapore,tapis_hydroskimming_singapore,x50_50_hls_lls_cracking_usgc,x30_70_wcs_bakken_cracking_usmc,bakken_coking_usmc,split
0,2018-01-01,Algeria,569.0,-0.412926,-0.434697,-0.499991,-0.615285,-0.394586,-1.001348,-0.569552,...,-0.292615,-0.774433,-0.251503,0.162508,0.579561,0.282312,0.149961,-0.184520,-0.171070,test
1,2018-02-01,Algeria,601.0,-0.548116,-0.560742,-0.565209,-0.290507,-0.014638,-0.239172,0.404461,...,-0.056544,-0.009556,0.774769,0.558664,1.048560,0.876053,0.870806,-0.622057,-0.628337,test
2,2018-03-01,Algeria,612.0,-0.509111,-0.541230,-0.539557,-0.948498,-0.521236,-0.540867,0.246635,...,0.029300,-0.002408,0.711157,0.512057,0.902313,0.637230,0.661418,-0.448739,-0.458053,test
3,2018-04-01,Algeria,640.0,-0.312954,-0.340651,-0.384773,-0.286289,-0.394586,0.086340,0.368386,...,0.115144,0.119115,0.503358,0.125223,0.297152,0.332067,0.304428,-0.002719,0.008741,test
4,2018-05-01,Algeria,519.0,-0.122856,-0.136559,-0.225642,-0.117574,-0.068916,-0.004962,0.417989,...,0.308294,-0.052446,0.482154,0.022688,0.413141,0.375188,0.441732,0.571775,0.599378,test
5,2018-06-01,Algeria,510.0,-0.178143,-0.153730,-0.332599,0.046924,0.442205,0.094280,0.832846,...,0.939248,-0.220433,0.656027,-0.690392,0.166034,0.335384,0.692311,0.436030,0.435047,test
6,2018-07-01,Algeria,430.0,-0.206545,-0.173241,-0.188250,0.055360,0.532669,0.300702,1.229667,...,1.063722,0.097670,1.181885,-0.387449,0.836754,0.375188,0.826182,0.188779,0.140920,test
7,2018-08-01,Algeria,630.0,-0.256152,-0.196265,-0.312164,1.033910,1.297089,0.753244,1.428077,...,1.801981,0.744599,1.699262,0.227757,1.088904,1.148045,1.430319,0.460270,0.439811,test
8,2018-09-01,Algeria,666.0,-0.039167,-0.005442,-0.215642,-0.429698,-0.245321,-0.552776,-0.055490,...,0.076515,-0.660059,-0.103075,-0.075185,0.372797,0.474697,0.520681,0.443302,0.458863,test
9,2018-10-01,Algeria,611.0,0.021800,0.070263,-0.192163,-1.281711,-0.381016,-0.588503,0.666002,...,-0.017914,-0.502795,0.376135,-0.261612,0.806496,0.000369,0.342187,0.021522,-0.025792,test


,bakken_coking_usmc,brent,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,dubai,dubai_cracking_singapore,dubai_hydroskimming_singapore,es_sider_cracking_med,es_sider_hydroskimming_med,tapis_hydroskimming_singapore,urals_cracking_med,urals_cracking_nw_europe,urals_hydroskimming_med,urals_hydroskimming_nw_europe,wti,x30_70_wcs_bakken_cracking_usmc,x50_50_hls_lls_cracking_usgc
Algeria,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
Denmark,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
Germany,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
Italy,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
Netherlands,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
Norway,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
Romania,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
Russian Federation,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
Turkey,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
United Kingdom,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


---

# Feature Selection with Per-Group Preprocessing

This section demonstrates how different feature selection methods can select **different features for each group** when using `per_group_prep=True`. This is powerful for grouped data where different variables may be important for different groups.

We'll demonstrate:
1. **step_select_permutation()** - Permutation importance-based selection
2. **step_select_shap()** - SHAP value-based selection
3. **step_safe_v2()** - SAFE feature engineering + selection
4. **step_filter_rf_importance()** - Random Forest importance filtering

**Key Point**: With `per_group_prep=True`, each group's recipe will select features independently based on that group's data patterns.

In [13]:
# Import feature selection steps
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

print("✓ Feature selection imports complete")

✓ Feature selection imports complete


In [14]:
# train_data_r = train_data.rename(columns={"refinery_kbd":'target'})
# train_data_r = train_data.rename(columns={"refinery_kbd":'target'})
# display(train_data_r.head(10))

## 1. Permutation Importance Feature Selection

`step_select_permutation()` uses permutation importance to rank features by how much model performance degrades when that feature is shuffled.

**How it works with grouped data**:
- Each group's recipe fits a model on that group's training data
- Calculates permutation importance for each feature
- Selects top features based on `top_n` or `top_p` threshold
- Different groups may select different features

In [15]:
# Create recipe with permutation importance feature selection
rec_perm = (
    recipe()
    .step_normalize()  # Normalize first
    .step_select_permutation(
        outcome='refinery_kbd',
        model=RandomForestRegressor(n_estimators=50, random_state=42),
        top_n=3,  # Select top 3 features per group
        n_repeats=5,  # Repeat permutation 5 times for stability
        random_state=42
    )
)

# Create workflow
wf_perm = (
    workflow()
    .add_recipe(rec_perm)
    .add_model(linear_reg().set_engine("sklearn"))
    .add_model_name("permutation_selection")
    .add_model_group_name("feature_selection_models")
)

# Fit with per-group preprocessing
print("Fitting model with permutation importance feature selection...")
fit_perm = wf_perm.fit_nested(train_data, group_col='country')
fit_perm = fit_perm.evaluate(test_data)

print("\n✓ Permutation importance model fitted successfully!")

fit = fit_perm

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

Fitting model with permutation importance feature selection...

✓ Permutation importance model fitted successfully!


,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,-0.411900,0.174585,-0.411900,-0.586485,train,permutation_selection,feature_selection_models,Algeria
1,2006-02-01,Algeria,-0.554261,0.214893,-0.554261,-0.769154,train,permutation_selection,feature_selection_models,Algeria
2,2006-03-01,Algeria,-0.625441,0.066308,-0.625441,-0.691750,train,permutation_selection,feature_selection_models,Algeria
3,2006-04-01,Algeria,-0.401731,-0.182672,-0.401731,-0.219060,train,permutation_selection,feature_selection_models,Algeria
4,2006-05-01,Algeria,-1.235559,0.032600,-1.235559,-1.268159,train,permutation_selection,feature_selection_models,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,-1.596937,-0.742647,-1.596937,-0.854290,test,permutation_selection,feature_selection_models,United Kingdom
1916,2021-09-01,United Kingdom,-2.022208,-0.782207,-2.022208,-1.240001,test,permutation_selection,feature_selection_models,United Kingdom
1917,2021-10-01,United Kingdom,-2.193722,-0.825896,-2.193722,-1.367826,test,permutation_selection,feature_selection_models,United Kingdom
1918,2021-11-01,United Kingdom,-1.539314,-1.322174,-1.539314,-0.217140,test,permutation_selection,feature_selection_models,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,-1.794460e-16,0.079857,-2.247083e-15,1.000000e+00,NaN,permutation_selection,feature_selection_models,Algeria,-0.157882,0.157882
1,Algeria,x50_50_hls_lls_cracking_usgc,1.467756e-01,0.093717,1.566150e+00,1.195710e-01,1.377246,permutation_selection,feature_selection_models,Algeria,-0.038509,0.332060
2,Algeria,bakken_coking_usmc,-2.991069e-01,0.084670,-3.532605e+00,5.577988e-04,1.124174,permutation_selection,feature_selection_models,Algeria,-0.466505,-0.131709
3,Algeria,wti,-2.446374e-02,0.095188,-2.570057e-01,7.975517e-01,1.420794,permutation_selection,feature_selection_models,Algeria,-0.212655,0.163727
4,Denmark,Intercept,2.385770e-17,0.080343,2.969476e-16,1.000000e+00,NaN,permutation_selection,feature_selection_models,Denmark,-0.158843,0.158843
5,Denmark,urals_cracking_nw_europe,2.999411e-01,0.098215,3.053916e+00,2.704412e-03,1.494379,permutation_selection,feature_selection_models,Denmark,0.105764,0.494118
6,Denmark,bakken_coking_usmc,-1.793596e-01,0.086871,-2.064670e+00,4.079960e-02,1.169097,permutation_selection,feature_selection_models,Denmark,-0.351108,-0.007611
7,Denmark,dubai_cracking_singapore,-1.703794e-01,0.104346,-1.632826e+00,1.047527e-01,1.686775,permutation_selection,feature_selection_models,Denmark,-0.376678,0.035919
8,Germany,Intercept,-2.024658e-17,0.059619,-3.395990e-16,1.000000e+00,NaN,permutation_selection,feature_selection_models,Germany,-0.117870,0.117870
9,Germany,urals_cracking_med,7.115992e-01,0.065284,1.090008e+01,0.000000e+00,1.199060,permutation_selection,feature_selection_models,Germany,0.582530,0.840669


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,0.944884,permutation_selection,feature_selection_models,Algeria
1,Algeria,train,mae,0.698023,permutation_selection,feature_selection_models,Algeria
2,Algeria,train,mape,335.34698,permutation_selection,feature_selection_models,Algeria
3,Algeria,train,smape,144.316024,permutation_selection,feature_selection_models,Algeria
4,Algeria,train,r_squared,0.107194,permutation_selection,feature_selection_models,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,permutation_selection,feature_selection_models,United Kingdom
256,United Kingdom,,model_class,LinearRegression,permutation_selection,feature_selection_models,United Kingdom
257,United Kingdom,train,n_obs_train,144,permutation_selection,feature_selection_models,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,permutation_selection,feature_selection_models,United Kingdom


,date,country,x50_50_hls_lls_cracking_usgc,bakken_coking_usmc,wti,refinery_kbd,urals_cracking_nw_europe,dubai_cracking_singapore,urals_cracking_med,dubai_hydroskimming_singapore,brent_hydroskimming_nw_europe,brent_cracking_nw_europe,dubai,es_sider_cracking_med,urals_hydroskimming_med,split
1430,2017-03-01,United Kingdom,1.097357,NaN,NaN,-1.261554,NaN,NaN,-0.131079,NaN,NaN,NaN,NaN,NaN,1.016494,train
1431,2017-04-01,United Kingdom,1.457779,NaN,NaN,-1.144334,NaN,NaN,0.158431,NaN,NaN,NaN,NaN,NaN,1.270942,train
1432,2017-05-01,United Kingdom,1.231228,NaN,NaN,-1.325671,NaN,NaN,0.058354,NaN,NaN,NaN,NaN,NaN,1.364239,train
1433,2017-06-01,United Kingdom,1.498971,NaN,NaN,-1.069161,NaN,NaN,0.133412,NaN,NaN,NaN,NaN,NaN,1.792559,train
1434,2017-07-01,United Kingdom,1.423454,NaN,NaN,-1.268952,NaN,NaN,0.340715,NaN,NaN,NaN,NaN,NaN,1.834967,train
1435,2017-08-01,United Kingdom,1.420021,NaN,NaN,-0.969327,NaN,NaN,0.433644,NaN,NaN,NaN,NaN,NaN,1.639891,train
1436,2017-09-01,United Kingdom,1.166009,NaN,NaN,-0.972327,NaN,NaN,0.912586,NaN,NaN,NaN,NaN,NaN,1.970672,train
1437,2017-10-01,United Kingdom,0.953188,NaN,NaN,-1.138251,NaN,NaN,0.144134,NaN,NaN,NaN,NaN,NaN,1.186126,train
1438,2017-11-01,United Kingdom,0.754098,NaN,NaN,-1.420820,NaN,NaN,-0.356253,NaN,NaN,NaN,NaN,NaN,0.562730,train
1439,2017-12-01,United Kingdom,0.551575,NaN,NaN,-1.329370,NaN,NaN,-0.677930,NaN,NaN,NaN,NaN,NaN,0.066557,train


,date,country,x50_50_hls_lls_cracking_usgc,bakken_coking_usmc,wti,refinery_kbd,urals_cracking_nw_europe,dubai_cracking_singapore,urals_cracking_med,dubai_hydroskimming_singapore,brent_hydroskimming_nw_europe,brent_cracking_nw_europe,dubai,es_sider_cracking_med,urals_hydroskimming_med,split
0,2018-01-01,Algeria,0.149961,-0.171070,-0.499991,0.798167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
1,2018-02-01,Algeria,0.870806,-0.628337,-0.565209,1.123563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
2,2018-03-01,Algeria,0.661418,-0.458053,-0.539557,1.235418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
3,2018-04-01,Algeria,0.304428,0.008741,-0.384773,1.520140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
4,2018-05-01,Algeria,0.441732,0.599378,-0.225642,0.289735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
5,2018-06-01,Algeria,0.692311,0.435047,-0.332599,0.198218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
6,2018-07-01,Algeria,0.826182,0.140920,-0.188250,-0.615273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
7,2018-08-01,Algeria,1.430319,0.439811,-0.312164,1.418453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
8,2018-09-01,Algeria,0.520681,0.458863,-0.215642,1.784524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
9,2018-10-01,Algeria,0.342187,-0.025792,-0.192163,1.225249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test


,bakken_coking_usmc,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,dubai,dubai_cracking_singapore,dubai_hydroskimming_singapore,es_sider_cracking_med,urals_cracking_med,urals_cracking_nw_europe,urals_hydroskimming_med,wti,x50_50_hls_lls_cracking_usgc
Algeria,True,False,False,False,False,False,False,False,False,False,True,True
Denmark,True,False,False,False,True,False,False,False,True,False,False,False
Germany,False,False,False,False,False,False,False,True,False,False,True,True
Italy,False,False,False,False,False,True,False,True,False,False,False,True
Netherlands,False,False,True,False,False,False,False,True,False,False,False,True
Norway,False,True,True,False,True,False,False,False,False,False,False,False
Romania,True,False,False,False,False,False,False,True,False,False,True,False
Russian Federation,False,False,False,True,False,False,False,True,False,False,False,True
Turkey,False,False,False,True,False,False,True,False,True,False,False,False
United Kingdom,False,False,False,False,False,False,False,True,False,True,False,True


## 2. SHAP Value Feature Selection

`step_select_shap()` uses SHAP (SHapley Additive exPlanations) values to determine feature importance based on game theory.

**How it works with grouped data**:
- Each group's recipe fits a model and calculates SHAP values
- Features are ranked by mean absolute SHAP value
- Top features are selected based on `top_n`, `top_p`, or `threshold`
- Groups may identify different important features

In [16]:
# Create recipe with SHAP-based feature selection
rec_shap = (
    recipe()
    .step_normalize()  # Normalize first
    .step_select_shap(
        outcome='refinery_kbd',
        model=GradientBoostingRegressor(n_estimators=50, random_state=42),
        top_n=4,  # Select top 4 features per group
        shap_samples=100,  # Use 100 samples for SHAP calculation (faster)
        random_state=42
    )
)

# Create workflow
wf_shap = (
    workflow()
    .add_recipe(rec_shap)
    .add_model(linear_reg().set_engine("sklearn"))
    .add_model_name("shap_selection")
    .add_model_group_name("feature_selection_models")
)

# Fit with per-group preprocessing
print("Fitting model with SHAP-based feature selection...")
fit_shap = wf_shap.fit_nested(train_data, group_col='country')
fit_shap = fit_shap.evaluate(test_data)

print("\n✓ SHAP-based model fitted successfully!")

fit = fit_shap

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

Fitting model with SHAP-based feature selection...

✓ SHAP-based model fitted successfully!


,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,-0.411900,-0.201132,-0.411900,-0.210768,train,shap_selection,feature_selection_models,Algeria
1,2006-02-01,Algeria,-0.554261,-0.292144,-0.554261,-0.262117,train,shap_selection,feature_selection_models,Algeria
2,2006-03-01,Algeria,-0.625441,-0.402080,-0.625441,-0.223361,train,shap_selection,feature_selection_models,Algeria
3,2006-04-01,Algeria,-0.401731,-0.543338,-0.401731,0.141606,train,shap_selection,feature_selection_models,Algeria
4,2006-05-01,Algeria,-1.235559,-0.450776,-1.235559,-0.784783,train,shap_selection,feature_selection_models,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,-1.596937,-0.672816,-1.596937,-0.924121,test,shap_selection,feature_selection_models,United Kingdom
1916,2021-09-01,United Kingdom,-2.022208,-0.674077,-2.022208,-1.348131,test,shap_selection,feature_selection_models,United Kingdom
1917,2021-10-01,United Kingdom,-2.193722,-0.665995,-2.193722,-1.527727,test,shap_selection,feature_selection_models,United Kingdom
1918,2021-11-01,United Kingdom,-1.539314,-1.285274,-1.539314,-0.254040,test,shap_selection,feature_selection_models,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,-2.391093e-16,0.075369,-3.172533e-15,1.000000e+00,NaN,shap_selection,feature_selection_models,Algeria,-0.149017,0.149017
1,Algeria,bakken_coking_usmc,-2.873632e-01,0.079959,-3.593900e+00,4.513184e-04,1.125510,shap_selection,feature_selection_models,Algeria,-0.445455,-0.129271
2,Algeria,x50_50_hls_lls_cracking_usgc,3.058908e-01,0.096003,3.186271e+00,1.779902e-03,1.622508,shap_selection,feature_selection_models,Algeria,0.116076,0.495705
3,Algeria,wti,4.907149e-02,0.091478,5.364279e-01,5.925203e-01,1.473178,shap_selection,feature_selection_models,Algeria,-0.131797,0.229940
4,Algeria,urals_cracking_med,-3.520268e-01,0.082579,-4.262917e+00,3.702317e-05,1.200485,shap_selection,feature_selection_models,Algeria,-0.515300,-0.188754
5,Denmark,Intercept,5.113713e-17,0.081880,6.245350e-16,1.000000e+00,NaN,shap_selection,feature_selection_models,Denmark,-0.161892,0.161892
6,Denmark,dubai_cracking_singapore,-3.873021e-02,0.090834,-4.263823e-01,6.704889e-01,1.230672,shap_selection,feature_selection_models,Denmark,-0.218326,0.140866
7,Denmark,bakken_coking_usmc,-1.668084e-01,0.099114,-1.682991e+00,9.462159e-02,1.465254,shap_selection,feature_selection_models,Denmark,-0.362775,0.029158
8,Denmark,es_sider_hydroskimming_med,-4.655844e-02,0.163023,-2.855941e-01,7.756142e-01,3.964051,shap_selection,feature_selection_models,Denmark,-0.368884,0.275767
9,Denmark,urals_hydroskimming_nw_europe,2.202955e-01,0.157808,1.395976e+00,1.649474e-01,3.714465,shap_selection,feature_selection_models,Denmark,-0.091718,0.532309


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,0.888582,shap_selection,feature_selection_models,Algeria
1,Algeria,train,mae,0.655038,shap_selection,feature_selection_models,Algeria
2,Algeria,train,mape,333.272674,shap_selection,feature_selection_models,Algeria
3,Algeria,train,smape,125.707671,shap_selection,feature_selection_models,Algeria
4,Algeria,train,r_squared,0.210421,shap_selection,feature_selection_models,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,shap_selection,feature_selection_models,United Kingdom
256,United Kingdom,,model_class,LinearRegression,shap_selection,feature_selection_models,United Kingdom
257,United Kingdom,train,n_obs_train,144,shap_selection,feature_selection_models,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,shap_selection,feature_selection_models,United Kingdom


,date,country,bakken_coking_usmc,x50_50_hls_lls_cracking_usgc,wti,urals_cracking_med,refinery_kbd,dubai_cracking_singapore,es_sider_hydroskimming_med,urals_hydroskimming_nw_europe,dubai,urals_hydroskimming_med,urals_cracking_nw_europe,brent_hydroskimming_nw_europe,brent_cracking_nw_europe,brent,es_sider_cracking_med,split
1430,2017-03-01,United Kingdom,NaN,1.097357,-1.112604,-0.131079,-1.261554,NaN,NaN,NaN,NaN,1.016494,NaN,NaN,NaN,NaN,NaN,train
1431,2017-04-01,United Kingdom,NaN,1.457779,-1.048256,0.158431,-1.144334,NaN,NaN,NaN,NaN,1.270942,NaN,NaN,NaN,NaN,NaN,train
1432,2017-05-01,United Kingdom,NaN,1.231228,-1.159561,0.058354,-1.325671,NaN,NaN,NaN,NaN,1.364239,NaN,NaN,NaN,NaN,NaN,train
1433,2017-06-01,United Kingdom,NaN,1.498971,-1.304345,0.133412,-1.069161,NaN,NaN,NaN,NaN,1.792559,NaN,NaN,NaN,NaN,NaN,train
1434,2017-07-01,United Kingdom,NaN,1.423454,-1.239997,0.340715,-1.268952,NaN,NaN,NaN,NaN,1.834967,NaN,NaN,NaN,NaN,NaN,train
1435,2017-08-01,United Kingdom,NaN,1.420021,-1.179996,0.433644,-0.969327,NaN,NaN,NaN,NaN,1.639891,NaN,NaN,NaN,NaN,NaN,train
1436,2017-09-01,United Kingdom,NaN,1.166009,-1.101735,0.912586,-0.972327,NaN,NaN,NaN,NaN,1.970672,NaN,NaN,NaN,NaN,NaN,train
1437,2017-10-01,United Kingdom,NaN,0.953188,-1.026517,0.144134,-1.138251,NaN,NaN,NaN,NaN,1.186126,NaN,NaN,NaN,NaN,NaN,train
1438,2017-11-01,United Kingdom,NaN,0.754098,-0.805211,-0.356253,-1.420820,NaN,NaN,NaN,NaN,0.562730,NaN,NaN,NaN,NaN,NaN,train
1439,2017-12-01,United Kingdom,NaN,0.551575,-0.749124,-0.677930,-1.329370,NaN,NaN,NaN,NaN,0.066557,NaN,NaN,NaN,NaN,NaN,train


,date,country,bakken_coking_usmc,x50_50_hls_lls_cracking_usgc,wti,urals_cracking_med,refinery_kbd,dubai_cracking_singapore,es_sider_hydroskimming_med,urals_hydroskimming_nw_europe,dubai,urals_hydroskimming_med,urals_cracking_nw_europe,brent_hydroskimming_nw_europe,brent_cracking_nw_europe,brent,es_sider_cracking_med,split
0,2018-01-01,Algeria,-0.171070,0.149961,-0.499991,-0.774433,0.798167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
1,2018-02-01,Algeria,-0.628337,0.870806,-0.565209,-0.009556,1.123563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
2,2018-03-01,Algeria,-0.458053,0.661418,-0.539557,-0.002408,1.235418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
3,2018-04-01,Algeria,0.008741,0.304428,-0.384773,0.119115,1.520140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
4,2018-05-01,Algeria,0.599378,0.441732,-0.225642,-0.052446,0.289735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
5,2018-06-01,Algeria,0.435047,0.692311,-0.332599,-0.220433,0.198218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
6,2018-07-01,Algeria,0.140920,0.826182,-0.188250,0.097670,-0.615273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
7,2018-08-01,Algeria,0.439811,1.430319,-0.312164,0.744599,1.418453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
8,2018-09-01,Algeria,0.458863,0.520681,-0.215642,-0.660059,1.784524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
9,2018-10-01,Algeria,-0.025792,0.342187,-0.192163,-0.502795,1.225249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test


,bakken_coking_usmc,brent,brent_cracking_nw_europe,brent_hydroskimming_nw_europe,dubai,dubai_cracking_singapore,es_sider_cracking_med,es_sider_hydroskimming_med,urals_cracking_med,urals_cracking_nw_europe,urals_hydroskimming_med,urals_hydroskimming_nw_europe,wti,x50_50_hls_lls_cracking_usgc
Algeria,True,False,False,False,False,False,False,False,True,False,False,False,True,True
Denmark,True,False,False,False,False,True,False,True,False,False,False,True,False,False
Germany,True,False,False,False,True,False,False,False,True,False,False,False,False,True
Italy,True,False,False,False,True,False,False,False,True,False,False,False,False,True
Netherlands,False,False,False,False,False,False,False,False,True,True,True,False,False,True
Norway,False,False,True,True,False,True,False,False,False,False,False,True,False,False
Romania,True,True,False,False,False,False,False,False,True,False,False,False,True,False
Russian Federation,False,True,False,False,False,False,False,False,True,False,True,False,False,True
Turkey,False,False,False,True,True,False,True,False,False,True,False,False,False,False
United Kingdom,False,False,False,False,False,False,False,False,True,False,True,False,True,True


## 3. SAFE Feature Engineering + Selection

`step_safe_v2()` performs **Surrogate Assisted Feature Extraction** - it creates interpretable transformations of features (thresholds, bins) and selects the most important ones.

**How it works with grouped data**:
- Each group's recipe fits a surrogate model
- Creates partial dependence plots to find optimal feature thresholds
- Generates binary features based on thresholds (e.g., `brent_gt_50`, `wti_lt_70`)
- Selects top transformed features based on importance
- Different groups may have different thresholds and transformations

**Note**: SAFE is computationally intensive, so we use `top_n` to limit features.

In [17]:
# Create recipe with SAFE v2 feature engineering + selection
rec_safe = (
    recipe()
    .step_normalize()  # Normalize first
    .step_safe_v2(
        surrogate_model=GradientBoostingRegressor(n_estimators=50, random_state=42),
        outcome='refinery_kbd',
        penalty=10.0,  # Changepoint penalty (higher = fewer thresholds)
        top_n=5,  # Select top 5 transformed features per group
        max_thresholds=3,  # Max 3 thresholds per feature
        keep_original_cols=False,  # Only keep transformed features
        feature_type='numeric'  # Only numeric features
    )
)

# Create workflow
wf_safe = (
    workflow()
    .add_recipe(rec_safe)
    .add_model(linear_reg().set_engine("sklearn"))
    .add_model_name("safe_selection")
    .add_model_group_name("feature_selection_models")
)

# Fit with per-group preprocessing
print("Fitting model with SAFE feature engineering + selection...")
print("(This may take a minute due to PDP calculations)")
fit_safe = wf_safe.fit_nested(train_data, group_col='country')
fit_safe = fit_safe.evaluate(test_data)

print("\n✓ SAFE-based model fitted successfully!")

fit = fit_safe

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

Fitting model with SAFE feature engineering + selection...
(This may take a minute due to PDP calculations)


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py_recipes/steps/feature_extraction.py:1226: UserWarning:

surrogate_model appears to be already fitted. step_safe_v2 expects UNFITTED model which will be fitted during prep().

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py_recipes/steps/feature_extraction.py:1226: UserWarning:

surrogate_model appears to be already fitted. step_safe_v2 expects UNFITTED model which will be fitted during prep().

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py_recipes/steps/feature_extraction.py:1226: UserWarning:

surrogate_model appears to be already fitted. step_safe_v2 expects UNFITTED model which will be fitted during prep().

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py_recipes/steps/feature_extraction.py:1226: UserWarning:

surrogate_model appears to be already fitted. step_safe_v2 expects UNFITTED model which will be fitted during p


✓ SAFE-based model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py_recipes/steps/feature_extraction.py:1226: UserWarning:

surrogate_model appears to be already fitted. step_safe_v2 expects UNFITTED model which will be fitted during prep().



,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,-0.411900,-0.415309,-0.411900,0.003409,train,safe_selection,feature_selection_models,Algeria
1,2006-02-01,Algeria,-0.554261,-0.075633,-0.554261,-0.478628,train,safe_selection,feature_selection_models,Algeria
2,2006-03-01,Algeria,-0.625441,-0.415309,-0.625441,-0.210132,train,safe_selection,feature_selection_models,Algeria
3,2006-04-01,Algeria,-0.401731,-0.415309,-0.401731,0.013578,train,safe_selection,feature_selection_models,Algeria
4,2006-05-01,Algeria,-1.235559,0.141358,-1.235559,-1.376917,train,safe_selection,feature_selection_models,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,-1.596937,-1.224743,-1.596937,-0.372195,test,safe_selection,feature_selection_models,United Kingdom
1916,2021-09-01,United Kingdom,-2.022208,-0.529237,-2.022208,-1.492971,test,safe_selection,feature_selection_models,United Kingdom
1917,2021-10-01,United Kingdom,-2.193722,-0.812220,-2.193722,-1.381503,test,safe_selection,feature_selection_models,United Kingdom
1918,2021-11-01,United Kingdom,-1.539314,-1.006664,-1.539314,-0.532650,test,safe_selection,feature_selection_models,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,0.247343,0.158930,1.556302,1.219268e-01,NaN,safe_selection,feature_selection_models,Algeria,-0.066910,0.561596
1,Algeria,bakken_coking_usmc_gt_1_57,-1.587785,0.255573,-6.212660,5.764259e-09,1.076168,safe_selection,feature_selection_models,Algeria,-2.093129,-1.082441
2,Algeria,urals_cracking_med_gt_0_16,-0.662652,0.161082,-4.113751,6.657210e-05,1.301360,safe_selection,feature_selection_models,Algeria,-0.981161,-0.344144
3,Algeria,x50_50_hls_lls_cracking_usgc_gt_0_17,0.216991,0.179118,1.211444,2.277957e-01,1.609083,safe_selection,feature_selection_models,Algeria,-0.137179,0.571161
4,Algeria,es_sider_hydroskimming_med_gt_0_14,0.339677,0.192435,1.765151,7.975036e-02,1.857249,safe_selection,feature_selection_models,Algeria,-0.040826,0.720179
5,Algeria,brent_gt_0_19,-0.102018,0.156964,-0.649943,5.168093e-01,1.235674,safe_selection,feature_selection_models,Algeria,-0.412384,0.208348
6,Denmark,Intercept,0.017394,0.141884,0.122594,9.026068e-01,NaN,safe_selection,feature_selection_models,Denmark,-0.263153,0.297941
7,Denmark,urals_cracking_nw_europe_gt_0_08,0.510274,0.219188,2.328020,2.136467e-02,1.982211,safe_selection,feature_selection_models,Denmark,0.076873,0.943676
8,Denmark,bakken_coking_usmc_gt_0_11,-0.362447,0.477161,-0.759590,4.487956e-01,9.393914,safe_selection,feature_selection_models,Denmark,-1.305940,0.581046
9,Denmark,x30_70_wcs_bakken_cracking_usmc_gt_0_11,-0.142697,0.476923,-0.299202,7.652358e-01,9.384543,safe_selection,feature_selection_models,Denmark,-1.085719,0.800326


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,0.829389,safe_selection,feature_selection_models,Algeria
1,Algeria,train,mae,0.598409,safe_selection,feature_selection_models,Algeria
2,Algeria,train,mape,353.677595,safe_selection,feature_selection_models,Algeria
3,Algeria,train,smape,118.223977,safe_selection,feature_selection_models,Algeria
4,Algeria,train,r_squared,0.312113,safe_selection,feature_selection_models,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,safe_selection,feature_selection_models,United Kingdom
256,United Kingdom,,model_class,LinearRegression,safe_selection,feature_selection_models,United Kingdom
257,United Kingdom,train,n_obs_train,144,safe_selection,feature_selection_models,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,safe_selection,feature_selection_models,United Kingdom


,date,country,bakken_coking_usmc_gt_1_57,urals_cracking_med_gt_0_16,x50_50_hls_lls_cracking_usgc_gt_0_17,es_sider_hydroskimming_med_gt_0_14,brent_gt_0_19,refinery_kbd,urals_cracking_nw_europe_gt_0_08,bakken_coking_usmc_gt_0_11,...,brent_cracking_nw_europe_gt_0_24,dubai_hydroskimming_singapore_gt_0_10,urals_cracking_med_gt_0_20,dubai_cracking_singapore_gt_0_10,es_sider_cracking_med_gt_0_84,dubai_gt_0_11,tapis_hydroskimming_singapore_gt_0_08,urals_cracking_med_gt_0_96,urals_hydroskimming_nw_europe_gt_0_07,split
1430,2017-03-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-1.261554,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,train
1431,2017-04-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-1.144334,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,train
1432,2017-05-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-1.325671,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,train
1433,2017-06-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-1.069161,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,train
1434,2017-07-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-1.268952,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,train
1435,2017-08-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-0.969327,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,train
1436,2017-09-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-0.972327,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,train
1437,2017-10-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-1.138251,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,train
1438,2017-11-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-1.420820,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,train
1439,2017-12-01,United Kingdom,NaN,NaN,NaN,NaN,NaN,-1.329370,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,train


,date,country,bakken_coking_usmc_gt_1_57,urals_cracking_med_gt_0_16,x50_50_hls_lls_cracking_usgc_gt_0_17,es_sider_hydroskimming_med_gt_0_14,brent_gt_0_19,refinery_kbd,urals_cracking_nw_europe_gt_0_08,bakken_coking_usmc_gt_0_11,...,brent_cracking_nw_europe_gt_0_24,dubai_hydroskimming_singapore_gt_0_10,urals_cracking_med_gt_0_20,dubai_cracking_singapore_gt_0_10,es_sider_cracking_med_gt_0_84,dubai_gt_0_11,tapis_hydroskimming_singapore_gt_0_08,urals_cracking_med_gt_0_96,urals_hydroskimming_nw_europe_gt_0_07,split
0,2018-01-01,Algeria,0.0,0.0,1.0,0.0,0.0,0.798167,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
1,2018-02-01,Algeria,0.0,1.0,1.0,1.0,0.0,1.123563,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
2,2018-03-01,Algeria,0.0,1.0,1.0,1.0,0.0,1.235418,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
3,2018-04-01,Algeria,0.0,1.0,1.0,1.0,0.0,1.520140,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
4,2018-05-01,Algeria,0.0,1.0,1.0,1.0,1.0,0.289735,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
5,2018-06-01,Algeria,0.0,0.0,1.0,1.0,1.0,0.198218,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
6,2018-07-01,Algeria,0.0,1.0,1.0,1.0,0.0,-0.615273,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
7,2018-08-01,Algeria,0.0,1.0,1.0,1.0,0.0,1.418453,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
8,2018-09-01,Algeria,0.0,0.0,1.0,1.0,1.0,1.784524,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
9,2018-10-01,Algeria,0.0,0.0,1.0,1.0,1.0,1.225249,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test


,bakken_coking_usmc_gt_0_11,bakken_coking_usmc_gt_1_57,brent_cracking_nw_europe_gt_0_18,brent_cracking_nw_europe_gt_0_24,brent_gt_0_19,brent_hydroskimming_nw_europe_gt_0_08,brent_hydroskimming_nw_europe_gt_0_12,dubai_cracking_singapore_gt_0_10,dubai_gt_0_11,dubai_gt_0_15,...,urals_cracking_med_gt_0_46,urals_cracking_med_gt_0_71,urals_cracking_med_gt_0_96,urals_cracking_nw_europe_gt_0_08,urals_hydroskimming_med_gt_0_05,urals_hydroskimming_nw_europe_gt_0_07,wti_gt_0_02,x30_70_wcs_bakken_cracking_usmc_gt_0_11,x50_50_hls_lls_cracking_usgc_gt_0_17,x50_50_hls_lls_cracking_usgc_gt_0_29
Algeria,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
Denmark,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,False,True,True,False
Germany,False,False,False,False,False,True,False,False,False,True,...,False,True,False,False,False,False,False,True,True,False
Italy,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,True,True,False,True
Netherlands,False,False,True,False,True,True,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
Norway,True,False,False,True,True,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
Romania,False,False,False,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
Russian Federation,False,False,False,False,False,False,False,False,False,False,...,False,True,False,True,False,False,True,True,False,True
Turkey,False,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,True,False,False,False
United Kingdom,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,False,True,False,False,False,True


## 4. Random Forest Importance Feature Selection

`step_filter_rf_importance()` uses Random Forest's built-in feature importance (mean decrease in impurity) to select features.

**How it works with grouped data**:
- Each group's recipe fits a Random Forest on that group's data
- Features are ranked by importance scores
- Top features are selected based on `top_n`, `top_p`, or `threshold`
- Fast and reliable for initial feature screening

In [18]:
# Create recipe with RF importance feature selection
rec_rf = (
    recipe()
    .step_normalize(all_numeric_predictors())  # Normalize first
    .step_filter_rf_importance(
        outcome='refinery_kbd',
        top_n=3  # Select top 3 features per group
        # n_estimators=100,  # RF trees
        # random_state=42
    )
)

# Create workflow
wf_rf = (
    workflow()
    .add_recipe(rec_rf)
    .add_model(linear_reg().set_engine("sklearn"))
    .add_model_name("rf_importance_selection")
    .add_model_group_name("feature_selection_models")
)

# Fit with per-group preprocessing
print("Fitting model with RF importance feature selection...")
fit_rf = wf_rf.fit_nested(train_data, group_col='country')
fit_rf = fit_rf.evaluate(test_data)

fit = fit_rf

outputs, coefs, stats = fit.extract_outputs()

display(outputs)
display(coefs)
display(stats)

# Plot forecast
fig = plot_forecast(fit, title="Complex Recipe")
fig.show()

# Extract preprocessed training data
processed_train = fit.extract_preprocessed_data(train_data, split='train')
processed_test = fit.extract_preprocessed_data(test_data, split='test')    

display(processed_train.tail(10))
display(processed_test.head(10))

display(fit.get_feature_comparison())

Fitting model with RF importance feature selection...


,date,country,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2006-01-01,Algeria,-0.411900,0.038419,-0.411900,-0.450319,train,rf_importance_selection,feature_selection_models,Algeria
1,2006-02-01,Algeria,-0.554261,0.082023,-0.554261,-0.636284,train,rf_importance_selection,feature_selection_models,Algeria
2,2006-03-01,Algeria,-0.625441,-0.017141,-0.625441,-0.608300,train,rf_importance_selection,feature_selection_models,Algeria
3,2006-04-01,Algeria,-0.401731,0.036129,-0.401731,-0.437860,train,rf_importance_selection,feature_selection_models,Algeria
4,2006-05-01,Algeria,-1.235559,-0.162693,-1.235559,-1.072866,train,rf_importance_selection,feature_selection_models,Algeria
...,...,...,...,...,...,...,...,...,...,...
1915,2021-08-01,United Kingdom,-1.596937,-0.809700,-1.596937,-0.787237,test,rf_importance_selection,feature_selection_models,United Kingdom
1916,2021-09-01,United Kingdom,-2.022208,-0.852128,-2.022208,-1.170080,test,rf_importance_selection,feature_selection_models,United Kingdom
1917,2021-10-01,United Kingdom,-2.193722,-0.863454,-2.193722,-1.330268,test,rf_importance_selection,feature_selection_models,United Kingdom
1918,2021-11-01,United Kingdom,-1.539314,-1.219203,-1.539314,-0.320111,test,rf_importance_selection,feature_selection_models,United Kingdom


,country,variable,coefficient,std_error,t_stat,p_value,vif,model,model_group_name,group,ci_0.025,ci_0.975
0,Algeria,Intercept,-2.326549e-15,0.080632,-2.885381e-14,1.000000e+00,NaN,rf_importance_selection,feature_selection_models,Algeria,-0.159414,0.159414
1,Algeria,brent,-2.788883e+00,1.289888,-2.162112e+00,3.230892e-02,255.909082,rf_importance_selection,feature_selection_models,Algeria,-5.339060,-0.238705
2,Algeria,es_sider_hydroskimming_med,-2.297564e-01,0.091078,-2.522636e+00,1.276640e-02,1.275874,rf_importance_selection,feature_selection_models,Algeria,-0.409822,-0.049690
3,Algeria,dubai,2.515682e+00,1.279462,1.966203e+00,5.125332e-02,251.788937,rf_importance_selection,feature_selection_models,Algeria,-0.013883,5.045248
4,Denmark,Intercept,-5.450483e-16,0.083234,-6.548404e-15,1.000000e+00,NaN,rf_importance_selection,feature_selection_models,Denmark,-0.164558,0.164558
5,Denmark,dubai,7.863905e-01,1.264161,6.220652e-01,5.349108e-01,230.677736,rf_importance_selection,feature_selection_models,Denmark,-1.712924,3.285705
6,Denmark,brent,-7.939661e-01,1.274864,-6.227847e-01,5.344391e-01,234.600472,rf_importance_selection,feature_selection_models,Denmark,-3.314441,1.726509
7,Denmark,urals_hydroskimming_med,1.542081e-01,0.107261,1.437685e+00,1.527550e-01,1.660688,rf_importance_selection,feature_selection_models,Denmark,-0.057854,0.366270
8,Germany,Intercept,2.746409e-15,0.070437,3.899111e-14,1.000000e+00,NaN,rf_importance_selection,feature_selection_models,Germany,-0.139257,0.139257
9,Germany,urals_cracking_med,4.079545e-01,0.072885,5.597262e+00,1.109274e-07,1.070713,rf_importance_selection,feature_selection_models,Germany,0.263858,0.552051


,country,split,metric,value,model,model_group_name,group
0,Algeria,train,rmse,0.954054,rf_importance_selection,feature_selection_models,Algeria
1,Algeria,train,mae,0.711539,rf_importance_selection,feature_selection_models,Algeria
2,Algeria,train,mape,267.061305,rf_importance_selection,feature_selection_models,Algeria
3,Algeria,train,smape,148.01972,rf_importance_selection,feature_selection_models,Algeria
4,Algeria,train,r_squared,0.08978,rf_importance_selection,feature_selection_models,Algeria
...,...,...,...,...,...,...,...
255,United Kingdom,,model_type,linear_reg,rf_importance_selection,feature_selection_models,United Kingdom
256,United Kingdom,,model_class,LinearRegression,rf_importance_selection,feature_selection_models,United Kingdom
257,United Kingdom,train,n_obs_train,144,rf_importance_selection,feature_selection_models,United Kingdom
258,United Kingdom,train,train_start_date,2006-01-01T00:00:00.000000000,rf_importance_selection,feature_selection_models,United Kingdom


,date,country,brent,es_sider_hydroskimming_med,dubai,refinery_kbd,urals_hydroskimming_med,urals_cracking_med,x50_50_hls_lls_cracking_usgc,brent_hydroskimming_nw_europe,urals_cracking_nw_europe,split
1430,2017-03-01,United Kingdom,-1.057443,NaN,NaN,-1.261554,NaN,-0.131079,1.097357,NaN,NaN,train
1431,2017-04-01,United Kingdom,-1.019196,NaN,NaN,-1.144334,NaN,0.158431,1.457779,NaN,NaN,train
1432,2017-05-01,United Kingdom,-1.099098,NaN,NaN,-1.325671,NaN,0.058354,1.231228,NaN,NaN,train
1433,2017-06-01,United Kingdom,-1.249813,NaN,NaN,-1.069161,NaN,0.133412,1.498971,NaN,NaN,train
1434,2017-07-01,United Kingdom,-1.181650,NaN,NaN,-1.268952,NaN,0.340715,1.423454,NaN,NaN,train
1435,2017-08-01,United Kingdom,-1.080163,NaN,NaN,-0.969327,NaN,0.433644,1.420021,NaN,NaN,train
1436,2017-09-01,United Kingdom,-0.936643,NaN,NaN,-0.972327,NaN,0.912586,1.166009,NaN,NaN,train
1437,2017-10-01,United Kingdom,-0.843487,NaN,NaN,-1.138251,NaN,0.144134,0.953188,NaN,NaN,train
1438,2017-11-01,United Kingdom,-0.656040,NaN,NaN,-1.420820,NaN,-0.356253,0.754098,NaN,NaN,train
1439,2017-12-01,United Kingdom,-0.593936,NaN,NaN,-1.329370,NaN,-0.677930,0.551575,NaN,NaN,train


,date,country,brent,es_sider_hydroskimming_med,dubai,refinery_kbd,urals_hydroskimming_med,urals_cracking_med,x50_50_hls_lls_cracking_usgc,brent_hydroskimming_nw_europe,urals_cracking_nw_europe,split
0,2018-01-01,Algeria,-0.412926,-0.292615,-0.434697,0.798167,NaN,NaN,NaN,NaN,NaN,test
1,2018-02-01,Algeria,-0.548116,-0.056544,-0.560742,1.123563,NaN,NaN,NaN,NaN,NaN,test
2,2018-03-01,Algeria,-0.509111,0.029300,-0.541230,1.235418,NaN,NaN,NaN,NaN,NaN,test
3,2018-04-01,Algeria,-0.312954,0.115144,-0.340651,1.520140,NaN,NaN,NaN,NaN,NaN,test
4,2018-05-01,Algeria,-0.122856,0.308294,-0.136559,0.289735,NaN,NaN,NaN,NaN,NaN,test
5,2018-06-01,Algeria,-0.178143,0.939248,-0.153730,0.198218,NaN,NaN,NaN,NaN,NaN,test
6,2018-07-01,Algeria,-0.206545,1.063722,-0.173241,-0.615273,NaN,NaN,NaN,NaN,NaN,test
7,2018-08-01,Algeria,-0.256152,1.801981,-0.196265,1.418453,NaN,NaN,NaN,NaN,NaN,test
8,2018-09-01,Algeria,-0.039167,0.076515,-0.005442,1.784524,NaN,NaN,NaN,NaN,NaN,test
9,2018-10-01,Algeria,0.021800,-0.017914,0.070263,1.225249,NaN,NaN,NaN,NaN,NaN,test


,brent,brent_hydroskimming_nw_europe,dubai,es_sider_hydroskimming_med,urals_cracking_med,urals_cracking_nw_europe,urals_hydroskimming_med,x50_50_hls_lls_cracking_usgc
Algeria,True,False,True,True,False,False,False,False
Denmark,True,False,True,False,False,False,True,False
Germany,True,False,True,False,True,False,False,False
Italy,True,False,True,False,True,False,False,False
Netherlands,True,False,True,False,False,False,False,True
Norway,True,True,True,False,False,False,False,False
Romania,True,False,True,False,True,False,False,False
Russian Federation,True,False,True,False,False,False,False,True
Turkey,True,False,True,False,False,True,False,False
United Kingdom,True,False,False,False,True,False,False,True


## Comparison of Feature Selection Methods

Let's compare how different feature selection methods performed and what features they selected for each group.

## Key Takeaways

### Per-Group Feature Selection Benefits

1. **Adaptive to Group Patterns**: Each group selects features most relevant to its data
2. **Improved Interpretability**: Different groups may have different drivers
3. **Better Performance**: Group-specific features can improve accuracy

### Method Selection Guide

| Method | Speed | Interpretability | Best For |
|--------|-------|------------------|----------|
| **step_filter_rf_importance()** | ⚡⚡⚡ Fast | ⭐⭐ Good | Initial screening, high-dimensional data |
| **step_select_permutation()** | ⚡⚡ Medium | ⭐⭐⭐ Very Good | Model-agnostic importance, any model type |
| **step_select_shap()** | ⚡ Slow | ⭐⭐⭐ Very Good | Detailed explanations, tree-based models |
| **step_safe_v2()** | 🐌 Very Slow | ⭐⭐⭐⭐ Excellent | Interpretable rules, threshold discovery |

### Recommendations

1. **Start with RF importance** for quick screening
2. **Use permutation importance** for robust, model-agnostic selection
3. **Use SHAP** for detailed explanations and feature interactions
4. **Use SAFE** when you need interpretable thresholds and rules

### Next Steps

- Try combining multiple selection methods in sequence
- Experiment with different `top_n` values per group
- Compare per-group vs shared preprocessing performance
- Use `.extract_preprocessed_data()` to inspect selected features